# DATA PROCESSING

# Импорт необходимых модумей

In [1]:
import re
import os
import logging
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from openpyxl import Workbook
from openpyxl.drawing.image import Image
from datetime import datetime
from dateutil.relativedelta import relativedelta


# Загрузка данных

In [2]:
# Избавляемся от не нужных предупреждений
warnings.filterwarnings('ignore')

# Настройка логирования для отслеживания выполнения кода
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Устанавливание директории где храниться датасет
os.chdir("source") 

# Отображать все столбцы
pd.set_option('display.max_columns', None)

# Отображать все строки
pd.set_option('display.max_rows', None)


In [3]:
# Загрузка данных из CSV файлов
data_app = pd.read_csv('ListPI1.csv')
data_loan = pd.read_csv('ListP1.csv')
data_beh = pd.read_csv('ListB1.csv')


# Объединение данных по Account ID

In [4]:
logging.info("Объединение данных по 'Account ID'.")
res = data_loan.merge(data_app, on="Account ID", how="left", suffixes=('_loan', '_app'))
res = res.merge(data_beh, on="Account ID", how="left", suffixes=('', '_beh'))


2024-12-12 13:01:24,476 - INFO - Объединение данных по 'Account ID'.


# Преобразование названий столбцов в snake_case

In [5]:
logging.info("Просмотр название столбцов до преобразования")
for i, column in enumerate(res.columns.to_list()):
    print(f"{i} <=> {column}")

2024-12-12 13:01:25,218 - INFO - Просмотр название столбцов до преобразования


0 <=> Customer ID_loan
1 <=> Application ID_loan
2 <=> Account ID
3 <=> Branch ID
4 <=> Product ID
5 <=> Application date
6 <=> Date loan granted
7 <=> Loan Amount
8 <=> First instalment due date
9 <=> Interest rate
10 <=> Collateral type
11 <=> Value of collateral
12 <=> Property type/Collateral type
13 <=> Salary payment in bank account
14 <=> Loan type
15 <=> # of instalments
16 <=> Instalment amount/Min instalment amount
17 <=> Customer ID_app
18 <=> Application ID_app
19 <=> Date of birth
20 <=> Gender
21 <=> City of Living
22 <=> Region of living
23 <=> City of registration
24 <=> Region of registration
25 <=> Work phone number
26 <=> Mobile phone number
27 <=> Education
28 <=> Marital status
29 <=> # Dependants
30 <=> # Children
31 <=> # months at current address
32 <=> Employment type
33 <=> Employment sector
34 <=> Employment segment
35 <=> # months at job
36 <=> Net main income
37 <=> Source of main income
38 <=> Additional income
39 <=> Source of additional income
40 <=> Rep

In [6]:
# Функция для преобразования названий столбцов в snake_case
def to_snake_case(column_name):
    return column_name.lower().replace('.', '_').replace('#', 'number').replace(' ', '_')

logging.info("Преобразование названий столбцов в snake_case.")

res.columns = [to_snake_case(col) for col in res.columns]


2024-12-12 13:01:25,223 - INFO - Преобразование названий столбцов в snake_case.


In [7]:
logging.info("Просмотр название столбцов после преобразования")
for i, column in enumerate(res.columns.to_list()):
    print(f"{i} <=> {column}")

2024-12-12 13:01:25,223 - INFO - Просмотр название столбцов после преобразования


0 <=> customer_id_loan
1 <=> application_id_loan
2 <=> account_id
3 <=> branch_id
4 <=> product_id
5 <=> application_date
6 <=> date_loan_granted
7 <=> loan_amount
8 <=> first_instalment_due_date
9 <=> interest_rate
10 <=> collateral_type
11 <=> value_of_collateral
12 <=> property_type/collateral_type
13 <=> salary_payment_in_bank_account
14 <=> loan_type
15 <=> number_of_instalments
16 <=> instalment_amount/min_instalment_amount
17 <=> customer_id_app
18 <=> application_id_app
19 <=> date_of_birth
20 <=> gender
21 <=> city_of_living
22 <=> region_of_living
23 <=> city_of_registration
24 <=> region_of_registration
25 <=> work_phone_number
26 <=> mobile_phone_number
27 <=> education
28 <=> marital_status
29 <=> number_dependants
30 <=> number_children
31 <=> number_months_at_current_address
32 <=> employment_type
33 <=> employment_sector
34 <=> employment_segment
35 <=> number_months_at_job
36 <=> net_main_income
37 <=> source_of_main_income
38 <=> additional_income
39 <=> source_of_add

# Удаление дублириющих переменных

In [8]:
res = res.drop(['application_id_app', 'customer_id_app', 'customer_id'], axis=1)
res = res.rename(columns={
    'customer_id_loan': 'customer_id',
    'application_id_loan': 'application_id'
})


# Переименование столбцов с русскими именами на английские

In [9]:
logging.info("Просмотр название столбцов после переименования")
res.head(2)

2024-12-12 13:01:26,899 - INFO - Просмотр название столбцов после переименования


,customer_id,application_id,account_id,branch_id,product_id,application_date,date_loan_granted,loan_amount,first_instalment_due_date,interest_rate,collateral_type,value_of_collateral,property_type/collateral_type,salary_payment_in_bank_account,loan_type,number_of_instalments,instalment_amount/min_instalment_amount,date_of_birth,gender,city_of_living,region_of_living,city_of_registration,region_of_registration,work_phone_number,mobile_phone_number,education,marital_status,number_dependants,number_children,number_months_at_current_address,employment_type,employment_sector,employment_segment,number_months_at_job,net_main_income,source_of_main_income,additional_income,source_of_additional_income,reported_expenses,months_with_bank,current_exposure,client_type_,property_object,наличие_эсхата_онлайн,наличие_пластиковых_карт,наличие_депозита,состояние,причина_отказа,run_date,date_account_opened,current_balance,date_last_payment,date_final_payment,due_date,payment_amount,account_status,number_of_payments_in_arrears,кумулятивная_просрочка,amount_due_–_instalment,principal_amount,interest_accrued,outstanding_balance,arrears_amount,current_days_past_due,maximum_days_past_due,maximum_days_past_due_lifetime,default_flag,кол-во_пролонгации,рейтинг_бки,количество_кредитов__в_бки_(заемщик),ftd-1,ftd-2,ftd-3,ftd-4
0,25121517058,764446/КР,35619143897,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ДУШАНБЕ",Карзхои гуногунмаксад,2021-10-19,2021-11-02,12300.0,2021-12-02,31.0,NaN,NaN,NaN,Нет,Многоцелевые кредиты_005_аннуитет,18,863.0,1998-01-07 00:00:00,Женский,нохияи Фирдавси,Душанбе,Фирдавси,Душанбе,992000805085,NaN,Высшее,Не замужем,0,2,0,Имеет другой источник дохода,NaN,NaN,0,3121.36,Прочее,NaN,NaN,1272.0,36,0.0,0,Квартира,Да,Да,0.0,NaN,NaN,2023-06-30,2021-11-02,0.0,NaN,2023-04-20,NaN,NaN,Закрыт,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,NaN,0,0,0,0,0
1,847140141,766801/КР,35733163635,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ИСТАРАВШАН",Карзхои гуногунмаксад,2021-10-25,2021-11-01,20000.0,2021-12-01,30.0,1) Поручитель;,1) 0;,1) Поручитель;,Нет,Многоцелевые кредиты_005_аннуитет,24,1118.0,1969-05-16 00:00:00,Мужской,Истаравшан,Вилояти Сугд,Истаравшан,Вилояти Сугд,992985675558,NaN,Среднее,Женат,2,2,6,Собственный бизнес,Самозанятость,Услуги транспорта,276,3800.00,Прочее,NaN,NaN,1500.0,143,4691.17,1,Дом,Нет,Нет,0.0,NaN,NaN,2023-06-30,2021-11-01,0.0,2023-04-26,2023-10-26,2023-06-01,1118.0,Закрыт,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,NaN,5,0,0,0,0


In [10]:
rename_columns = {
    'наличие_эсхата_онлайн': 'eskhata_online',
    'наличие_пластиковых_карт': 'plastic_cards',
    'наличие_депозита': 'deposit',
    'кумулятивная_просрочка': 'cumulative_delinquency',
    'рейтинг_бки': 'bki_rating',
    'количество_кредитов__в_бки_(заемщик)': 'bki_number_of_loans',
    'состояние': 'state',
    'причина_отказа': 'rejection_reason',
    'кол-во_пролонгации': 'number_of_extensions'
    
}
logging.info("Переименование столбцов с русскими именами на английские.")
res.rename(columns=rename_columns, inplace=True)


2024-12-12 13:01:27,526 - INFO - Переименование столбцов с русскими именами на английские.


# Переименование столбцов для удобства

In [11]:
rename_columns = {
    'client_type_': 'client_type',
    'number_dependants':'dependants',
    'current_days_past_due': 'current_dpd',
    'number_months_at_current_address':'months_at_current_address',
    'number_months_at_job':'months_at_job',
    'property_type/collateral_type': 'property_type',
    'instalment_amount/min_instalment_amount': 'instalment_amount',
    'amount_due_–_instalment': 'amount_due',
    'maximum_days_past_due': 'max_dpd',
    'maximum_days_past_due_lifetime': 'max_dpd_lifetime',
    'number_of_payments_in_arrears': 'payments_in_arrears',
    'ftd-1': 'ftd_1',
    'ftd-2': 'ftd_2',
    'ftd-3': 'ftd_3',
    'ftd-4': 'ftd_4'
}

logging.info("Переименование столбцов")
res.rename(columns=rename_columns, inplace=True)

2024-12-12 13:01:28,857 - INFO - Переименование столбцов


In [12]:
logging.info("Просмотр название столбцов после переименования")
res.head(2)

2024-12-12 13:01:29,488 - INFO - Просмотр название столбцов после переименования


,customer_id,application_id,account_id,branch_id,product_id,application_date,date_loan_granted,loan_amount,first_instalment_due_date,interest_rate,collateral_type,value_of_collateral,property_type,salary_payment_in_bank_account,loan_type,number_of_instalments,instalment_amount,date_of_birth,gender,city_of_living,region_of_living,city_of_registration,region_of_registration,work_phone_number,mobile_phone_number,education,marital_status,dependants,number_children,months_at_current_address,employment_type,employment_sector,employment_segment,months_at_job,net_main_income,source_of_main_income,additional_income,source_of_additional_income,reported_expenses,months_with_bank,current_exposure,client_type,property_object,eskhata_online,plastic_cards,deposit,state,rejection_reason,run_date,date_account_opened,current_balance,date_last_payment,date_final_payment,due_date,payment_amount,account_status,payments_in_arrears,cumulative_delinquency,amount_due,principal_amount,interest_accrued,outstanding_balance,arrears_amount,current_dpd,max_dpd,max_dpd_lifetime,default_flag,number_of_extensions,bki_rating,bki_number_of_loans,ftd_1,ftd_2,ftd_3,ftd_4
0,25121517058,764446/КР,35619143897,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ДУШАНБЕ",Карзхои гуногунмаксад,2021-10-19,2021-11-02,12300.0,2021-12-02,31.0,NaN,NaN,NaN,Нет,Многоцелевые кредиты_005_аннуитет,18,863.0,1998-01-07 00:00:00,Женский,нохияи Фирдавси,Душанбе,Фирдавси,Душанбе,992000805085,NaN,Высшее,Не замужем,0,2,0,Имеет другой источник дохода,NaN,NaN,0,3121.36,Прочее,NaN,NaN,1272.0,36,0.0,0,Квартира,Да,Да,0.0,NaN,NaN,2023-06-30,2021-11-02,0.0,NaN,2023-04-20,NaN,NaN,Закрыт,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,NaN,0,0,0,0,0
1,847140141,766801/КР,35733163635,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ИСТАРАВШАН",Карзхои гуногунмаксад,2021-10-25,2021-11-01,20000.0,2021-12-01,30.0,1) Поручитель;,1) 0;,1) Поручитель;,Нет,Многоцелевые кредиты_005_аннуитет,24,1118.0,1969-05-16 00:00:00,Мужской,Истаравшан,Вилояти Сугд,Истаравшан,Вилояти Сугд,992985675558,NaN,Среднее,Женат,2,2,6,Собственный бизнес,Самозанятость,Услуги транспорта,276,3800.00,Прочее,NaN,NaN,1500.0,143,4691.17,1,Дом,Нет,Нет,0.0,NaN,NaN,2023-06-30,2021-11-01,0.0,2023-04-26,2023-10-26,2023-06-01,1118.0,Закрыт,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,NaN,5,0,0,0,0


# Подсчет количества дубликатов (не включая первую строку)

In [13]:
logging.info("Проверка дублирующихся строк по account_id")
display(res.duplicated(subset='account_id').sum())


logging.info("Проверка дублирующихся строк по customer_id")
display(res.duplicated(subset='customer_id').sum())


logging.info("Подсчет количества дубликатов по application_id")
display(res.duplicated(subset='application_id').sum())


2024-12-12 13:01:30,883 - INFO - Проверка дублирующихся строк по account_id


0

2024-12-12 13:01:30,891 - INFO - Проверка дублирующихся строк по customer_id


60704

2024-12-12 13:01:30,898 - INFO - Подсчет количества дубликатов по application_id


0

# Удаление дубликатов

In [14]:
# Удаление дубликатов по account_id
logging.info("Удаление дубликатов по account_id.")
res.drop_duplicates(subset='account_id', keep='first', inplace=True)


# Удаление дубликатов по application_id
logging.info("Удаление дубликатов по application_id.")
res.drop_duplicates(subset='application_id', keep='first', inplace=True)


# Удаление дубликатов по customer_id
# logging.info("Удаление дубликатов по customer_id.")
# res.drop_duplicates(subset='customer_id', keep='first', inplace=True)

2024-12-12 13:01:32,907 - INFO - Удаление дубликатов по account_id.
2024-12-12 13:01:33,049 - INFO - Удаление дубликатов по application_id.


In [15]:
# Создание копии датафрейма для дальнейшей обработки
df = res.copy()

In [16]:
df.head(2)

,customer_id,application_id,account_id,branch_id,product_id,application_date,date_loan_granted,loan_amount,first_instalment_due_date,interest_rate,collateral_type,value_of_collateral,property_type,salary_payment_in_bank_account,loan_type,number_of_instalments,instalment_amount,date_of_birth,gender,city_of_living,region_of_living,city_of_registration,region_of_registration,work_phone_number,mobile_phone_number,education,marital_status,dependants,number_children,months_at_current_address,employment_type,employment_sector,employment_segment,months_at_job,net_main_income,source_of_main_income,additional_income,source_of_additional_income,reported_expenses,months_with_bank,current_exposure,client_type,property_object,eskhata_online,plastic_cards,deposit,state,rejection_reason,run_date,date_account_opened,current_balance,date_last_payment,date_final_payment,due_date,payment_amount,account_status,payments_in_arrears,cumulative_delinquency,amount_due,principal_amount,interest_accrued,outstanding_balance,arrears_amount,current_dpd,max_dpd,max_dpd_lifetime,default_flag,number_of_extensions,bki_rating,bki_number_of_loans,ftd_1,ftd_2,ftd_3,ftd_4
0,25121517058,764446/КР,35619143897,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ДУШАНБЕ",Карзхои гуногунмаксад,2021-10-19,2021-11-02,12300.0,2021-12-02,31.0,NaN,NaN,NaN,Нет,Многоцелевые кредиты_005_аннуитет,18,863.0,1998-01-07 00:00:00,Женский,нохияи Фирдавси,Душанбе,Фирдавси,Душанбе,992000805085,NaN,Высшее,Не замужем,0,2,0,Имеет другой источник дохода,NaN,NaN,0,3121.36,Прочее,NaN,NaN,1272.0,36,0.0,0,Квартира,Да,Да,0.0,NaN,NaN,2023-06-30,2021-11-02,0.0,NaN,2023-04-20,NaN,NaN,Закрыт,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,NaN,0,0,0,0,0
1,847140141,766801/КР,35733163635,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ИСТАРАВШАН",Карзхои гуногунмаксад,2021-10-25,2021-11-01,20000.0,2021-12-01,30.0,1) Поручитель;,1) 0;,1) Поручитель;,Нет,Многоцелевые кредиты_005_аннуитет,24,1118.0,1969-05-16 00:00:00,Мужской,Истаравшан,Вилояти Сугд,Истаравшан,Вилояти Сугд,992985675558,NaN,Среднее,Женат,2,2,6,Собственный бизнес,Самозанятость,Услуги транспорта,276,3800.00,Прочее,NaN,NaN,1500.0,143,4691.17,1,Дом,Нет,Нет,0.0,NaN,NaN,2023-06-30,2021-11-01,0.0,2023-04-26,2023-10-26,2023-06-01,1118.0,Закрыт,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,NaN,5,0,0,0,0


In [17]:
# Анализ пропущенных значений
missing_percent = df.isnull().mean() * 100
logging.info("Процент пропущенных значений по столбцам (после удаления полностью пустых столбцов):")

# Вывод переменных с пропуском по убыванию
print(missing_percent.sort_values(ascending=False)[missing_percent > 0])
print("\n")

2024-12-12 13:01:34,989 - INFO - Процент пропущенных значений по столбцам (после удаления полностью пустых столбцов):


rejection_reason               100.000000
state                          100.000000
default_flag                    99.286375
payments_in_arrears             94.963046
mobile_phone_number             93.830240
property_type                   88.766557
value_of_collateral             88.766557
collateral_type                 88.766557
source_of_additional_income     77.727431
additional_income               77.544994
bki_rating                      62.926287
employment_segment              37.151355
employment_sector               37.084320
due_date                        20.328980
payment_amount                  20.328980
education                        5.172297
date_last_payment                4.887610
work_phone_number                4.362362
instalment_amount                0.512520
source_of_main_income            0.451425
employment_type                  0.124312
marital_status                   0.116675
net_main_income                  0.015274
current_exposure                 0

# Преобразование необходимых столбцов в числовые значения

In [18]:
numeric_columns = [
    'net_main_income', 'additional_income', 'months_with_bank',
    'reported_expenses', 'deposit', 'current_exposure'
]
missing_numeric = [col for col in numeric_columns if col not in df.columns]
if missing_numeric:
    logging.warning(f"Отсутствуют следующие числовые столбцы: {missing_numeric}")
    

logging.info("Просмотр столбцов с числами")
display(df[numeric_columns].head())

logging.info("Просмотр типа столбцов с числами до преобразования")
display(df[numeric_columns].dtypes)


2024-12-12 13:01:36,912 - INFO - Просмотр столбцов с числами


,net_main_income,additional_income,months_with_bank,reported_expenses,deposit,current_exposure
0,3121.36,NaN,36,1272.0,0.0,0.0
1,3800.00,NaN,143,1500.0,0.0,4691.17
2,3500.00,NaN,61,1000.0,0.0,4301.47
3,2400.00,NaN,60,1000.0,0.0,1153.52
4,5700.00,NaN,20,3001.0,0.0,4938.37


2024-12-12 13:01:36,925 - INFO - Просмотр типа столбцов с числами до преобразования


net_main_income      float64
additional_income    float64
months_with_bank      object
reported_expenses    float64
deposit               object
current_exposure      object
dtype: object

In [19]:
existing_numeric_columns = [col for col in numeric_columns if col in df.columns]
logging.info("Преобразование числовых столбцов в числовые значения.")
df[existing_numeric_columns] = df[existing_numeric_columns].apply(pd.to_numeric, errors='coerce')


2024-12-12 13:01:37,402 - INFO - Преобразование числовых столбцов в числовые значения.


In [20]:
logging.info("Просмотр типа столбцов с числами после преобразования")
display(df[numeric_columns].dtypes)

2024-12-12 13:01:37,811 - INFO - Просмотр типа столбцов с числами после преобразования


net_main_income      float64
additional_income    float64
months_with_bank       int64
reported_expenses    float64
deposit              float64
current_exposure     float64
dtype: object

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235698 entries, 0 to 235697
Data columns (total 74 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   customer_id                     235698 non-null  int64  
 1   application_id                  235698 non-null  object 
 2   account_id                      235698 non-null  int64  
 3   branch_id                       235698 non-null  object 
 4   product_id                      235698 non-null  object 
 5   application_date                235698 non-null  object 
 6   date_loan_granted               235698 non-null  object 
 7   loan_amount                     235698 non-null  float64
 8   first_instalment_due_date       235688 non-null  object 
 9   interest_rate                   235698 non-null  float64
 10  collateral_type                 26477 non-null   object 
 11  value_of_collateral             26477 non-null   object 
 12  property_type   

# Преобразование столбцов с датами в формат datetime

In [22]:
date_columns = [
    'application_date', 'date_of_birth', 'date_loan_granted', 'first_instalment_due_date',
    'date_last_payment', 'date_final_payment', 'due_date', 'date_account_opened'
]

logging.info("Просмотр столбцов с датами")
display(df[date_columns].head())

logging.info("Просмотр типа столбцов с датами до преобразования")
display(df[date_columns].dtypes)


2024-12-12 13:01:39,676 - INFO - Просмотр столбцов с датами


,application_date,date_of_birth,date_loan_granted,first_instalment_due_date,date_last_payment,date_final_payment,due_date,date_account_opened
0,2021-10-19,1998-01-07 00:00:00,2021-11-02,2021-12-02,NaN,2023-04-20,NaN,2021-11-02
1,2021-10-25,1969-05-16 00:00:00,2021-11-01,2021-12-01,2023-04-26,2023-10-26,2023-06-01,2021-11-01
2,2021-10-25,1991-03-02 00:00:00,2021-11-05,2021-12-06,2023-04-03,2023-10-26,2023-06-05,2021-11-05
3,2021-10-25,1966-10-23 00:00:00,2021-11-03,2021-12-03,2022-09-12,2022-10-26,NaN,2021-11-03
4,2021-10-26,2001-06-07 00:00:00,2021-11-03,2021-12-03,2022-04-27,2022-04-27,NaN,2021-11-03


2024-12-12 13:01:39,699 - INFO - Просмотр типа столбцов с датами до преобразования


application_date             object
date_of_birth                object
date_loan_granted            object
first_instalment_due_date    object
date_last_payment            object
date_final_payment           object
due_date                     object
date_account_opened          object
dtype: object

In [23]:
logging.info("Преобразование столбцов с датами в формат datetime.")
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')


2024-12-12 13:01:40,450 - INFO - Преобразование столбцов с датами в формат datetime.


In [24]:
logging.info("Просмотр типа столбцов с датами после преобразования")
display(df[date_columns].dtypes)


2024-12-12 13:01:41,071 - INFO - Просмотр типа столбцов с датами после преобразования


application_date             datetime64[ns]
date_of_birth                datetime64[ns]
date_loan_granted            datetime64[ns]
first_instalment_due_date    datetime64[ns]
date_last_payment            datetime64[ns]
date_final_payment           datetime64[ns]
due_date                     datetime64[ns]
date_account_opened          datetime64[ns]
dtype: object

# Исправление опечатки в тексте

In [25]:
df['eskhata_online'] = df['eskhata_online'].replace({'нет': 'Нет'})

# Вычисление возраста и исправление неправилных значений

In [26]:
logging.info("Вычисление возраста.")
df['age'] = df['application_date'].dt.year - df['date_of_birth'].dt.year


2024-12-12 13:01:42,619 - INFO - Вычисление возраста.


In [27]:
logging.info("Подсчет количество возрастов меньше 18 до корректировки")
display(df.query('age < 18').age.count())

2024-12-12 13:01:42,793 - INFO - Подсчет количество возрастов меньше 18 до корректировки


4

In [28]:
logging.info("Корректировка неправилных возрастов")

# Корректировка возраста, если месяц или день рождения еще не наступили в году заявки (но такие были исправлены вручную)
df.loc[
    (df['application_date'].dt.month < df['date_of_birth'].dt.month) |
    ((df['application_date'].dt.month == df['date_of_birth'].dt.month) &
     (df['application_date'].dt.day < df['date_of_birth'].dt.day)),
    'age'
] -= 1

# Исправление некоторых возрастов (которые были проверены и исправлены вручную)
df.loc[df['age'] == 17, 'age'] = 18
df.loc[df['age'] == 0, 'age'] = 27
df.loc[df['age'] == -1, 'age'] = 33
df['age'] = df['age'].fillna(df['age'].mean())

# Удаление отрицательных возрастов, если они остались
df['age'] = df['age'].where(df['age'] >= 0, np.nan)

# Приведение возраста к целочисленному типу
df.age = np.floor(df.age).astype(np.int64)


2024-12-12 13:01:42,998 - INFO - Корректировка неправилных возрастов


In [29]:
logging.info("Подсчет количество возрастов меньше 18 после корректировки")
display(df.query('age < 18').age.count())

2024-12-12 13:01:43,520 - INFO - Подсчет количество возрастов меньше 18 после корректировки


0

# Создание столбцов необходимых для дальнейших вычисленый

In [30]:
# Создание столбца месяц/год выдачи кредита как год * 100 + месяц
logging.info("Создание столбца год/месяц выдачи кредита как год.")
df['loan_month'] = df['application_date'].dt.year * 100 + df['application_date'].dt.month

# Признак совпадения города проживания и регистрации
logging.info("Создание признака совпадения города проживания и регистрации.")
df['city_of_living_eq_registration'] = (df['city_of_living'] == df['city_of_registration'])

# Признак наличия залога
logging.info("Создание признака наличия залога.")
df['is_collateral'] = np.where((df['deposit'].notna()) & (df['deposit'] != 0), True, False)

2024-12-12 13:01:44,064 - INFO - Создание столбца год/месяц выдачи кредита как год.
2024-12-12 13:01:44,076 - INFO - Создание признака совпадения города проживания и регистрации.
2024-12-12 13:01:44,098 - INFO - Создание признака наличия залога.


In [31]:
new_tables = [
    'age',
    'loan_month',
    'city_of_living_eq_registration',
    'is_collateral'
]

logging.info("Просмотр новых таблиц")
display(df[new_tables].head(5))
display(df[new_tables].dtypes)

2024-12-12 13:01:44,384 - INFO - Просмотр новых таблиц


,age,loan_month,city_of_living_eq_registration,is_collateral
0,23,202110,False,False
1,52,202110,True,False
2,30,202110,True,False
3,55,202110,False,False
4,20,202110,True,False


age                               int64
loan_month                        int32
city_of_living_eq_registration     bool
is_collateral                      bool
dtype: object

# Создание временного датафрейма для анализа предыдущих заявок

In [32]:
logging.info("Создание временного DataFrame для анализа предыдущих заявок.")
temp_df = df.copy()

# Сортировка по customer_id и application_date для корректного сдвига
logging.info("Сортировка по 'customer_id' и 'application_date'.")
temp_df = temp_df.sort_values(['customer_id', 'application_date'])

# Добавление предыдущей даты заявки и предыдущей максимальной просрочки
logging.info("Добавление предыдущей даты заявки и предыдущей максимальной просрочки.")
temp_df['prev_application_date'] = temp_df.groupby('customer_id')['application_date'].shift(1)
temp_df['prev_max_dpd'] = temp_df.groupby('customer_id')['max_dpd_lifetime'].shift(1)


# Убираем записи без предыдущих заявок
logging.info("Удаление записей без предыдущих заявок.")
temp_df = temp_df.dropna(subset=['prev_application_date'])

2024-12-12 13:01:45,097 - INFO - Создание временного DataFrame для анализа предыдущих заявок.
2024-12-12 13:01:45,572 - INFO - Сортировка по 'customer_id' и 'application_date'.
2024-12-12 13:01:45,840 - INFO - Добавление предыдущей даты заявки и предыдущей максимальной просрочки.
2024-12-12 13:01:45,877 - INFO - Удаление записей без предыдущих заявок.


# Подсчет количества предыдущих заявок и максимальной просрочки

In [33]:
logging.info("Группировка по 'account_id' для подсчета количества предыдущих заявок и максимальной просрочки.")
acc_numb_before = temp_df.groupby('account_id').agg(
    cnt=('customer_id', 'count'),
    max_mdpd=('prev_max_dpd', 'max')
).reset_index()

logging.info("Пример данных после группировки:")
print("\nacc_numb_before.head():")
print(acc_numb_before.head())

logging.info("Добавление флага существующего клиента.")
acc_numb_before['existing_client'] = np.where(acc_numb_before['cnt'] > 1, 1, 0)


2024-12-12 13:01:46,870 - INFO - Группировка по 'account_id' для подсчета количества предыдущих заявок и максимальной просрочки.
2024-12-12 13:01:46,896 - INFO - Пример данных после группировки:



acc_numb_before.head():
    account_id  cnt  max_mdpd
0  36002196446    1       4.0
1  36003105046    1      17.0
2  36031268759    1       0.0
3  36054772668    1       3.0
4  36060574691    1       0.0


2024-12-12 13:01:46,897 - INFO - Добавление флага существующего клиента.


In [34]:
# Объединение с основным датафреймом
logging.info("Объединение 'acc_numb_before' с основным DataFrame.")
df = df.merge(
    acc_numb_before[['account_id', 'cnt', 'max_mdpd']], 
    on='account_id', 
    how='left'
)

2024-12-12 13:02:00,130 - INFO - Объединение 'acc_numb_before' с основным DataFrame.


In [35]:
# Проверка наличия столбца cnt после объединения
logging.info("Проверка наличия столбца cnt после объединения.")
print("\nСтолбцы после объединения acc_numb_before:")
for column in df.columns.tolist():
    print(column)


2024-12-12 13:02:01,164 - INFO - Проверка наличия столбца cnt после объединения.



Столбцы после объединения acc_numb_before:
customer_id
application_id
account_id
branch_id
product_id
application_date
date_loan_granted
loan_amount
first_instalment_due_date
interest_rate
collateral_type
value_of_collateral
property_type
salary_payment_in_bank_account
loan_type
number_of_instalments
instalment_amount
date_of_birth
gender
city_of_living
region_of_living
city_of_registration
region_of_registration
work_phone_number
mobile_phone_number
education
marital_status
dependants
number_children
months_at_current_address
employment_type
employment_sector
employment_segment
months_at_job
net_main_income
source_of_main_income
additional_income
source_of_additional_income
reported_expenses
months_with_bank
current_exposure
client_type
property_object
eskhata_online
plastic_cards
deposit
state
rejection_reason
run_date
date_account_opened
current_balance
date_last_payment
date_final_payment
due_date
payment_amount
account_status
payments_in_arrears
cumulative_delinquency
amount_due


In [36]:
if 'cnt' not in df.columns:
    logging.error("Столбец 'cnt' отсутствует в DataFrame после объединения. Проверьте корректность группировки и объединения.")
    raise KeyError("Столбец 'cnt' отсутствует в DataFrame после объединения.")
else:
    # Заполнение пропусков в previous_loans_count нулями
    logging.info("Заполнение пропусков в 'previous_loans_count' нулями.")
    df['previous_loans_count'] = df['cnt'].fillna(0).astype(int)
    
    # Создание бинарных признаков для просроченности
    logging.info("Создание бинарных признаков для просроченности.")
    df.loc[:, 'gb_90ever'] = (df['max_dpd_lifetime'] >= 90).astype(int)
    df.loc[:, 'gb_cum_dlq_90'] = (df['cumulative_delinquency'] >= 90).astype(int)
    df.loc[:, 'gb_60ever'] = (df['max_dpd_lifetime'] >= 60).astype(int)
    
    # Удаление временных столбцов
    logging.info("Удаление временных столбцов 'cnt' и 'max_mdpd'.")
    df.drop(['cnt', 'max_mdpd'], axis=1, inplace=True)

2024-12-12 13:02:01,968 - INFO - Заполнение пропусков в 'previous_loans_count' нулями.
2024-12-12 13:02:01,971 - INFO - Создание бинарных признаков для просроченности.
2024-12-12 13:02:01,975 - INFO - Удаление временных столбцов 'cnt' и 'max_mdpd'.


# Структура датафрейма после создания новых переменных

In [37]:
logging.info("Просмотр структуры создания датафрейма после новых .")
display(df.info())

2024-12-12 13:02:03,377 - INFO - Просмотр структуры создания датафрейма после новых .


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235698 entries, 0 to 235697
Data columns (total 82 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   customer_id                     235698 non-null  int64         
 1   application_id                  235698 non-null  object        
 2   account_id                      235698 non-null  int64         
 3   branch_id                       235698 non-null  object        
 4   product_id                      235698 non-null  object        
 5   application_date                235698 non-null  datetime64[ns]
 6   date_loan_granted               235698 non-null  datetime64[ns]
 7   loan_amount                     235698 non-null  float64       
 8   first_instalment_due_date       235688 non-null  datetime64[ns]
 9   interest_rate                   235698 non-null  float64       
 10  collateral_type                 26477 non-null   object 

# Сохрание данных после созданые новых переменных

In [38]:
# Сохрание данных для дальнейшей работы
# df.to_csv('full_data.csv')

# Выборка 2% случайных строк для анализа
df_analysis = df.sample(frac=0.02, random_state=42)

# Сохраняю пример данных для дальнейшего анализа
# df_analysis.to_excel('sample_data.xlsx', index=False)


In [39]:
# Устанавливание директории где будуть храниться excel файлы результатов
os.chdir("../result_excel") 

# Выбор необходимых столбцов

In [40]:
include_vars = [
    # "customer_id",
    # "account_id",
#     "branch_id",
    "loan_amount",
    "is_collateral",
    "salary_payment_in_bank_account",
    "age",
    "gender",
#     "region_of_living",
    "region_of_registration",
    "city_of_living_eq_registration",
    "education",
    "marital_status",
    "dependants",
    "months_at_current_address",
    "employment_type",
    "employment_segment",
    "months_at_job",
    "net_main_income",
    "source_of_main_income",
    "additional_income",
    "reported_expenses",
    "months_with_bank",
    "client_type",
    "property_object",
    "eskhata_online",
    "plastic_cards",
#     "deposit",
    "gb_90ever",
    "gb_cum_dlq_90",
    "gb_60ever",
    "bki_rating",
    "bki_number_of_loans",
    "loan_month",
    "previous_loans_count",
]


In [41]:
# Проверка наличия всех столбцов
missing_columns = [col for col in include_vars if col not in df.columns]
if missing_columns:
    logging.warning(f"Отсутствуют следующие столбцы: {missing_columns}")

In [42]:
# Создание нового датафрейма с выбранными столбцами (только существующие)
existing_include_vars = [col for col in include_vars if col in df.columns]
df_selected = df[existing_include_vars].copy()

In [43]:
df_selected.head(6)

,loan_amount,is_collateral,salary_payment_in_bank_account,age,gender,region_of_registration,city_of_living_eq_registration,education,marital_status,dependants,months_at_current_address,employment_type,employment_segment,months_at_job,net_main_income,source_of_main_income,additional_income,reported_expenses,months_with_bank,client_type,property_object,eskhata_online,plastic_cards,gb_90ever,gb_cum_dlq_90,gb_60ever,bki_rating,bki_number_of_loans,loan_month,previous_loans_count
0,12300.0,False,Нет,23,Женский,Душанбе,False,Высшее,Не замужем,0,0,Имеет другой источник дохода,NaN,0,3121.36,Прочее,NaN,1272.0,36,0,Квартира,Да,Да,0,0,0,NaN,0,202110,0
1,20000.0,False,Нет,52,Мужской,Вилояти Сугд,True,Среднее,Женат,2,6,Собственный бизнес,Услуги транспорта,276,3800.00,Прочее,NaN,1500.0,143,1,Дом,Нет,Нет,0,0,0,NaN,5,202110,0
2,10000.0,False,Нет,30,Мужской,Вилояти Сугд,True,Среднее,Женат,3,9,Собственный бизнес,Услуги транспорта,60,3500.00,Прочее,NaN,1000.0,61,1,Дом,Нет,Нет,0,0,0,NaN,2,202110,0
3,3300.0,False,Нет,55,Мужской,Вилояти Хатлон,False,Среднее,Женат,1,39,Имеет другой источник дохода,NaN,0,2400.00,Доход семьи,NaN,1000.0,60,1,Дом,Нет,Нет,0,0,0,NaN,0,202110,0
4,5000.0,False,Нет,20,Мужской,Нохияхои тобеи Чумхури,True,Среднее,Холост,0,2,Собственный бизнес,Услуги транспорта,36,5700.00,Предпринимательство,NaN,3001.0,20,0,Дом,Да,Нет,0,0,0,NaN,1,202110,0
5,12000.0,False,Нет,55,Женский,Вилояти Хатлон,True,Среднее,Замужем,3,12,Имеет другой источник дохода,NaN,0,2161.00,Предпринимательство,NaN,1100.0,116,1,Дом,Нет,Нет,0,0,0,F,5,202110,0


# Вычисляем количество пропущенных значений в каждом столбце до исправления

In [44]:
missing_values = df_selected.isnull().sum()

columns_with_missing = missing_values.sort_values(ascending=False)[missing_values > 0]
columns_with_missing

additional_income        182772
bki_rating               148316
employment_segment        87565
education                 12191
source_of_main_income      1064
employment_type             293
marital_status              275
net_main_income              36
property_object              14
reported_expenses             8
dtype: int64

In [45]:
df_selected[columns_with_missing.keys().to_list()].head(10)

,additional_income,bki_rating,employment_segment,education,source_of_main_income,employment_type,marital_status,net_main_income,property_object,reported_expenses
0,NaN,NaN,NaN,Высшее,Прочее,Имеет другой источник дохода,Не замужем,3121.36,Квартира,1272.0
1,NaN,NaN,Услуги транспорта,Среднее,Прочее,Собственный бизнес,Женат,3800.00,Дом,1500.0
2,NaN,NaN,Услуги транспорта,Среднее,Прочее,Собственный бизнес,Женат,3500.00,Дом,1000.0
3,NaN,NaN,NaN,Среднее,Доход семьи,Имеет другой источник дохода,Женат,2400.00,Дом,1000.0
4,NaN,NaN,Услуги транспорта,Среднее,Предпринимательство,Собственный бизнес,Холост,5700.00,Дом,3001.0
5,NaN,F,NaN,Среднее,Предпринимательство,Имеет другой источник дохода,Замужем,2161.00,Дом,1100.0
6,NaN,NaN,Агро,Среднее специальное,Предпринимательство,Собственный бизнес,Женат,4000.00,Дом,2400.0
7,1095.0,F,Услуги,Среднее специальное,Предпринимательство,Собственный бизнес,Замужем,1360.00,Квартира,1138.0
8,NaN,NaN,Экономист,Высшее,Зарплата по основному месту работы,Работает в организации,Женат,3523.00,Квартира,1696.0
9,600.0,NaN,Прочие,NaN,Зарплата по основному месту работы,Работает в организации,Женат,2580.00,Дом,1576.0


In [46]:
# Столбцы с типом object, которые содержать пропуски
object_columns = ['bki_rating', 'employment_segment', 'education', 'source_of_main_income', 
                  'employment_type', 'marital_status', 'property_object']

for column in object_columns:
    # Заполняем пропущенные значения в столбце значением 'missing'
    df_selected[column] = df_selected[column].fillna('missing')



# Преобразование строковых переменных в категориальные

In [47]:
# Преобразуем столбцы с типом object, которые содержать пропуски  в категориальный тип
df_selected['additional_income'] = (df_selected['additional_income'] > 0).astype('bool')
df_selected['client_type'] = (df_selected['client_type'] > 0).astype('bool')
df_selected['gb_90ever'] = (df_selected['gb_90ever'] > 0).astype('int')
df_selected['gb_cum_dlq_90'] = (df_selected['gb_cum_dlq_90'] > 0).astype('int')
df_selected['gb_60ever'] = (df_selected['gb_60ever'] > 0).astype('int')


In [48]:
logging.info("Преобразование строковых переменных в категориальные.")
object_columns = df_selected.select_dtypes(include=['object']).columns

logging.info("Преобразование булевых переменных в категориальные.")
category_columns = df_selected.select_dtypes(include=['bool']).columns

for column in category_columns:
    # Преобразуем True в 'Да', False в 'Нет'
    df_selected[column] = df_selected[column].replace({True: 'Да', False: 'Нет'}).astype('category')

df_selected[object_columns] = df_selected[object_columns].astype('category')

2024-12-12 13:02:08,039 - INFO - Преобразование строковых переменных в категориальные.
2024-12-12 13:02:08,094 - INFO - Преобразование булевых переменных в категориальные.


In [49]:
# Вывод типов данных всех столбцов
print(df_selected.dtypes)

loan_amount                        float64
is_collateral                     category
salary_payment_in_bank_account    category
age                                  int64
gender                            category
region_of_registration            category
city_of_living_eq_registration    category
education                         category
marital_status                    category
dependants                           int64
months_at_current_address            int64
employment_type                   category
employment_segment                category
months_at_job                        int64
net_main_income                    float64
source_of_main_income             category
additional_income                 category
reported_expenses                  float64
months_with_bank                     int64
client_type                       category
property_object                   category
eskhata_online                    category
plastic_cards                     category
gb_90ever  

# Вычисляем количество пропущенных значений в каждом столбце до исправления

In [50]:
missing_values = df_selected.isnull().sum()

columns_with_missing = missing_values.sort_values(ascending=False)[missing_values > 0]
columns_with_missing

net_main_income      36
reported_expenses     8
dtype: int64

In [51]:
missing_numeric_columns = [
    'net_main_income', 
    'reported_expenses'
]

# Заполнение пропущенных значений

In [52]:
# Заполнение пропущенных значений в числовых столбцах медианой
for col in missing_numeric_columns:
    median = df_selected[col].median()
    df_selected[col].fillna(median, inplace=True)
    print(f"Заполнение пропусков в числовом столбце '{col}' медианой: {median}")

print("\n")

Заполнение пропусков в числовом столбце 'net_main_income' медианой: 2787.0
Заполнение пропусков в числовом столбце 'reported_expenses' медианой: 1137.0




# Определение типов столбцов

In [53]:
numeric_columns = df_selected.select_dtypes(include=['int64', 'int32', 'float64']).columns.tolist()
categorical_columns = df_selected.select_dtypes(include=['category', 'object']).columns.tolist()
datetime_columns = df_selected.select_dtypes(include=['datetime64[ns]']).columns.tolist()

print(f"Числовые столбцы: {numeric_columns}\n")
print(f"Категориальные столбцы: {categorical_columns}\n")
print(f"Временные столбцы: {datetime_columns}\n")

Числовые столбцы: ['loan_amount', 'age', 'dependants', 'months_at_current_address', 'months_at_job', 'net_main_income', 'reported_expenses', 'months_with_bank', 'gb_90ever', 'gb_cum_dlq_90', 'gb_60ever', 'bki_number_of_loans', 'loan_month', 'previous_loans_count']

Категориальные столбцы: ['is_collateral', 'salary_payment_in_bank_account', 'gender', 'region_of_registration', 'city_of_living_eq_registration', 'education', 'marital_status', 'employment_type', 'employment_segment', 'source_of_main_income', 'additional_income', 'client_type', 'property_object', 'eskhata_online', 'plastic_cards', 'bki_rating']

Временные столбцы: []



# Обработка выбросов

In [54]:
logging.info("Обработка выбросов для числовых столбцов")
# for col in numeric_columns:
#     lower_quantile = df_cleaned[col].quantile(0.01)
#     upper_quantile = df_cleaned[col].quantile(0.99)
#     original_min = df_cleaned[col].min()
#     original_max = df_cleaned[col].max()
    
#     # Обрезка выбросов
#     df_cleaned[col] = np.where(df_cleaned[col] < lower_quantile, lower_quantile, df_cleaned[col])
#     df_cleaned[col] = np.where(df_cleaned[col] > upper_quantile, upper_quantile, df_cleaned[col])
    
#     print(f"Обработка выбросов в столбце '{col}':")
#     print(f" - Оригинальный минимум: {original_min}, обрезанный минимум: {lower_quantile}")
#     print(f" - Оригинальный максимум: {original_max}, обрезанный максимум: {upper_quantile}\n")


2024-12-12 13:02:08,421 - INFO - Обработка выбросов для числовых столбцов


# Удаление дублей

In [55]:
duplicates = df_selected.duplicated().sum()
print(f"Количество дубликатов в DataFrame: {duplicates}")
if duplicates > 0:
    df_selected = df_selected.drop_duplicates()
    print(f"Дубликаты удалены. Новый размер DataFrame: {df_selected.shape}\n")
else:
    print("Дубликаты отсутствуют.\n")

Количество дубликатов в DataFrame: 12
Дубликаты удалены. Новый размер DataFrame: (235686, 30)



# Вывод сводных статистик после очистки

In [56]:
# Сводные статистики для числовых столбцов
print("Сводные статистики для числовых столбцов после очистки:")
display(df_selected[numeric_columns].describe())
print("\n")

Сводные статистики для числовых столбцов после очистки:


,loan_amount,age,dependants,months_at_current_address,months_at_job,net_main_income,reported_expenses,months_with_bank,gb_90ever,gb_cum_dlq_90,gb_60ever,bki_number_of_loans,loan_month,previous_loans_count
count,235686.000000,235686.000000,235686.000000,235686.000000,235686.000000,2.356860e+05,235686.000000,235686.000000,235686.000000,235686.000000,235686.000000,235686.000000,235686.000000,235686.000000
mean,10611.250368,38.833847,2.099993,16.439492,62.042557,5.173113e+03,1338.074489,43.151740,0.007544,0.030197,0.013488,2.871397,202229.647607,0.257512
std,12836.967409,12.371003,1.514300,144.212307,102.079039,4.147324e+05,836.074303,39.940815,0.086528,0.171129,0.115353,12.512430,58.604330,0.437265
min,500.000000,18.000000,0.000000,0.000000,0.000000,1.000000e-02,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,202110.000000,0.000000
25%,3500.000000,29.000000,1.000000,0.000000,0.000000,1.950000e+03,1000.000000,14.000000,0.000000,0.000000,0.000000,0.000000,202204.000000,0.000000
50%,6000.000000,37.000000,2.000000,3.000000,24.000000,2.787000e+03,1137.000000,29.000000,0.000000,0.000000,0.000000,1.000000,202209.000000,0.000000
75%,12000.000000,48.000000,3.000000,10.000000,96.000000,4.000000e+03,1500.000000,63.000000,0.000000,0.000000,0.000000,3.000000,202302.000000,1.000000
max,476000.000000,94.000000,34.000000,7191.000000,25440.000000,9.004431e+07,72800.000000,272.000000,1.000000,1.000000,1.000000,966.000000,202306.000000,1.000000


In [57]:
# 2) Сводные статистики для категориальных столбцов
print("Сводные статистики для категориальных столбцов после очистки:")
display(df_selected[categorical_columns].describe())
print("\n")

Сводные статистики для категориальных столбцов после очистки:


,is_collateral,salary_payment_in_bank_account,gender,region_of_registration,city_of_living_eq_registration,education,marital_status,employment_type,employment_segment,source_of_main_income,additional_income,client_type,property_object,eskhata_online,plastic_cards,bki_rating
count,235686,235686,235686,235686,235686,235686,235686,235686,235686,235686,235686,235686,235686,235686,235686,235686
unique,2,2,2,5,2,8,9,4,57,7,2,2,8,2,2,11
top,Нет,Нет,Мужской,Вилояти Сугд,Да,Среднее,Женат,Имеет другой источник дохода,missing,Предпринимательство,Нет,Да,Дом,Нет,Нет,missing
freq,231962,225042,122792,103782,194532,147127,99522,86709,87563,88529,182761,141491,201081,120139,147285,148311


In [58]:
# 3) Сводные статистики для временных столбцов
print("Сводные статистики для временных столбцов после очистки:")
# display(df_selected[datetime_columns].describe())
# print("\n")

Сводные статистики для временных столбцов после очистки:


# Визуализация сводных статистик

In [59]:
# Настройки для красивых графиков
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


# Функции для построения графиков

In [60]:
# Функция для построения графиков числовых столбцов
def plot_numeric(df, column):
    plt.figure(figsize=(14, 6))

    # Гистограмма
    plt.subplot(1, 2, 1)
    sns.histplot(df[column].dropna(), bins=30, color='skyblue', kde=True)
    plt.title(f"Гистограмма '{column}'")
    plt.xlabel(column)
    plt.ylabel('Частота')

    plt.tight_layout()
    plt.show()
    plt.close()

# Функция для построения графиков категориальных столбцов
def plot_categorical(df, column):
    plt.figure(figsize=(10, 6))
    # Замена пропущенных значений на 'Missing'
    data = df[column].astype(str).fillna('Missing')
    sns.countplot(y=data, palette="Set2", order=data.value_counts().index)
    plt.title(f"Распределение по '{column}'")
    plt.xlabel('Частота')
    plt.ylabel(column)
    plt.tight_layout()
    plt.show()
    plt.close()

# Функция для построения графиков временных столбцов
def plot_datetime(df, column):
    plt.figure(figsize=(14, 6))
    
    # Извлечение года из даты
    data_year = df[column].dropna().dt.year
    sns.histplot(data_year, bins=30, color='coral', kde=True)
    plt.title(f"Распределение по годам для '{column}'")
    plt.xlabel('Год')
    plt.ylabel('Частота')
    
    plt.tight_layout()
    plt.show()
    plt.close()


# Итерация по числовым столбцам и построение графиков

In [61]:
# Выборка 5% случайных строк для ускорения визуализации
sample_frac = 0.05
sample_df = df_selected.sample(frac=sample_frac, random_state=42)

# Определение типов столбцов
numeric_columns = sample_df.select_dtypes(include=['int64', 'int32', 'float64']).columns.tolist()
categorical_columns = sample_df.select_dtypes(include=['category', 'object']).columns.tolist()
datetime_columns = sample_df.select_dtypes(include=['datetime64[ns]']).columns.tolist()

In [62]:
print("Построение графиков для числовых столбцов...")
# for col in numeric_columns:
#     print(f"Построение для '{col}'...")
#     plot_numeric(sample_df, col)

Построение графиков для числовых столбцов...


# Итерация по категориальным столбцам и построение графиков

In [63]:
print("Построение графиков для категориальных столбцов...")
# for col in categorical_columns:
#     print(f"Построение для '{col}'...")
#     plot_categorical(sample_df, col)

Построение графиков для категориальных столбцов...


# Итерация по временным столбцам и построение графиков

In [64]:
print("Построение графиков для временных столбцов...")
# for col in datetime_columns:
#     print(f"Построение для '{col}'...")
#     plot_datetime(sample_df, col)

print("Все графики построены.")


Построение графиков для временных столбцов...
Все графики построены.


# Сохранение графиков

In [65]:
# Создаём папки для графиков
plot_folder = '../plots'
subfolders = ['numeric', 'categorical', 'datetime']
for subfolder in subfolders:
    os.makedirs(os.path.join(plot_folder, subfolder), exist_ok=True)

# Функция для построения графика распределения числовых данных
# с использованием категорий, основанных на статистических оценках
def plot_numeric(df, column, plot_folder, bins=5, method='quantile', display=False):
    try:
        bin_column = f'{column}_bin'

        if method == 'quantile':
            # Квартильное разбиение
            df[bin_column] = pd.qcut(df[column], q=bins, duplicates='drop')
        elif method == 'std':
            # Разбиение на основе стандартного отклонения
            mean = df[column].mean()
            std = df[column].std()
            edges = [float('-inf'), mean - std, mean, mean + std, float('inf')]
            labels = [f'< {mean - std:.1f}', f'{mean - std:.1f} - {mean:.1f}',
                      f'{mean:.1f} - {mean + std:.1f}', f'> {mean + std:.1f}']
            df[bin_column] = pd.cut(df[column], bins=edges, labels=labels)
        else:
            # Обычное равномерное разбиение
            df[bin_column] = pd.cut(df[column], bins=bins)
            df[bin_column] = df[bin_column].apply(
                lambda x: f"[{int(x.left)}, {int(x.right)}]" if pd.notnull(x) else "Unknown"
            )

        # Построение графика
        plt.figure(figsize=(14, 7))
        sns.countplot(y=bin_column, data=df, palette='viridis', order=df[bin_column].value_counts().index)
        plt.title(f'Распределение {column} по категориям ({method})', fontsize=16)
        plt.xlabel('Количество', fontsize=14)
        plt.ylabel(f'{column} (Бины)', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()

        save_path = os.path.join(plot_folder, 'numeric', f'{column}_binned_{method}.png')
        if display:
            plt.show()
        else:
            plt.savefig(save_path, dpi=300)
            plt.close()
            print(f"График для '{column}' успешно сохранён по пути: {save_path}")
    except Exception as e:
        print(f"Ошибка при создании графика для '{column}': {e}")

# Функция для создания графиков для категориальных столбцов
def plot_categorical(df, column, plot_folder, display=False):
    try:
        plt.figure(figsize=(14, 8))  # Увеличенный размер графика
        value_counts = df[column].value_counts().sort_values(ascending=True)
        sns.barplot(x=value_counts.values, y=value_counts.index, palette='viridis')
        plt.title(f'Гистограмма для {column}', fontsize=16)
        plt.xlabel('Частота', fontsize=14)
        plt.ylabel(column, fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()

        save_path = os.path.join(plot_folder, 'categorical', f'{column}.png')
        if display:
            plt.show()
        else:
            plt.savefig(save_path, dpi=300)
            plt.close()
            print(f"Гистограмма для '{column}' успешно сохранена по пути: {save_path}")
    except Exception as e:
        print(f"Ошибка при создании гистограммы для '{column}': {e}")

# Функция для создания графиков для временных данных
def plot_datetime(df, column, plot_folder, y_column, display=False):
    try:
        plt.figure(figsize=(16, 9))  # Увеличенный размер графика
        sns.lineplot(x=column, y=y_column, data=df, marker='o', label=y_column)
        plt.title(f'График {y_column} по дате {column}', fontsize=16)
        plt.xlabel('Дата', fontsize=14)
        plt.ylabel(y_column, fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xticks(rotation=45)
        plt.legend()

        # Форматирование дат на оси X
        plt.gcf().autofmt_xdate()

        plt.tight_layout()
        save_path = os.path.join(plot_folder, 'datetime', f'{column}_{y_column}.png')
        if display:
            plt.show()
        else:
            plt.savefig(save_path, dpi=300)
            plt.close()
            print(f"График для '{column}' по '{y_column}' успешно сохранён по пути: {save_path}")
    except Exception as e:
        print(f"Ошибка при создании графика для '{column}': {e}")

# Выбор режима: display=True для отображения графиков в Jupyter, display=False для сохранения в файлы
display_mode = False

# Строим графики для числовых столбцов с категоризацией
print("Построение графиков для числовых столбцов...")
for col in numeric_columns:
    plot_numeric(df_selected, col, plot_folder, bins=5, method='quantile', display=display_mode)

# Строим графики для категориальных столбцов
print("Построение графиков для категориальных столбцов...")
for col in categorical_columns:
    plot_categorical(df_selected, col, plot_folder, display=display_mode)

# Строим графики для временных столбцов
print("Построение графиков для временных столбцов...")
y_column_for_datetime = 'loan_amount'  # Укажите нужный числовой столбец для отображения по дате
for col in datetime_columns:
    plot_datetime(df_selected, col, plot_folder, y_column_for_datetime, display=display_mode)

print("Графики успешно созданы!")

Построение графиков для числовых столбцов...
График для 'loan_amount' успешно сохранён по пути: ../plots/numeric/loan_amount_binned_quantile.png
График для 'age' успешно сохранён по пути: ../plots/numeric/age_binned_quantile.png
График для 'dependants' успешно сохранён по пути: ../plots/numeric/dependants_binned_quantile.png
График для 'months_at_current_address' успешно сохранён по пути: ../plots/numeric/months_at_current_address_binned_quantile.png
График для 'months_at_job' успешно сохранён по пути: ../plots/numeric/months_at_job_binned_quantile.png
График для 'net_main_income' успешно сохранён по пути: ../plots/numeric/net_main_income_binned_quantile.png
График для 'reported_expenses' успешно сохранён по пути: ../plots/numeric/reported_expenses_binned_quantile.png
График для 'months_with_bank' успешно сохранён по пути: ../plots/numeric/months_with_bank_binned_quantile.png
График для 'gb_90ever' успешно сохранён по пути: ../plots/numeric/gb_90ever_binned_quantile.png
График для 'gb_

# Группировка по loan_month и вычисление показателей просроченности

In [66]:
logging.info("Группировка по 'loan_month' и вычисление показателей просроченности.")
rep_gb_90ever_month = df_selected.groupby('loan_month').agg(
    count=('gb_90ever', 'count'),
    bad_num=('gb_90ever', 'sum'),
    bad_rate=('gb_90ever', 'mean')
).reset_index()

rep_gb_cum_dlq_90_month = df_selected.groupby('loan_month').agg(
    count=('gb_cum_dlq_90', 'count'),
    bad_num=('gb_cum_dlq_90', 'sum'),
    bad_rate=('gb_cum_dlq_90', 'mean')
).reset_index()

rep_gb_60ever_month = df_selected.groupby('loan_month').agg(
    count=('gb_60ever', 'count'),
    bad_num=('gb_60ever', 'sum'),
    bad_rate=('gb_60ever', 'mean')
).reset_index()


2024-12-12 13:02:33,211 - INFO - Группировка по 'loan_month' и вычисление показателей просроченности.


In [67]:
rep_gb_60ever_month

,loan_month,count,bad_num,bad_rate
0,202110,277,9,0.032491
1,202111,9413,240,0.025497
2,202112,12723,400,0.031439
3,202201,11061,319,0.028840
4,202202,11555,358,0.030982
5,202203,5793,145,0.025030
6,202204,11325,250,0.022075
7,202205,10770,191,0.017734
8,202206,11572,239,0.020653
9,202207,12542,244,0.019455


# Создание выборок по типу занятости

In [68]:
df_selected.employment_segment.unique()

['missing', 'Услуги транспорта', 'Агро', 'Услуги', 'Экономист', ..., 'производство бумаги', 'Бахчевые', 'менеральные удобрения', 'рыбный промысел', 'Производство из пиломатериалов']
Length: 57
Categories (57, object): ['missing', 'Агро', 'Арендодатель', 'Бахчевые', ...,
                          'менеральные удобрения', 'производство бумаги', 'производство одежды',
                          'рыбный промысел']

In [69]:
def create_employment_samples(df):
    logging.info("Создание выборок по типу занятости.")
    
    # Определение категорий для выборки работников организаций
    employment_segments_empl = [
        "Мед. работник",
        "Работник в сфере образования",
        "Работник госструктур",
        "Работник НПО (Ташкилоти Чамъияти)",
        "Работник производства",
        "Работник сельского хозяйства",
        "Работник частной организации",
        "Строитель",
        "Экономист",
        "Услуги в системе образования",
        "Жилищно-коммунальные услуги",
        "Бытовые услуги",
        "Домашние животные",
        "Продукты питания",
        "Услуги связи",
        "Общепит",
        "Производство строительных материалов",
        "Производство из железных материалов",
        "Растеневодство",
        "производство одежды",
        "Производство из стекла",
        "Садоводства",
        "Овощеводства",
        "Строительные материалы",
        "Пищевая промышленность",
        "Хоз. товары",
        "Косметика и парфюмерия",
        "Пчеловодство",
        "ГСМ",
        "Галантерея",
        "Сырьевые материалы",
        "Мебель",
        "Арендодатель",
        "Медикаменты",
        "Бытовая техника",
        "Золото",
        "Средства связи",
        "Финансовые услуги",
        "Переработка хлопка",
        "Производство из пластмассы",
        "производство бумаги",
        "Бахчевые",
        "менеральные удобрения",
        "рыбный промысел",
        "Производство из пиломатериалов"
    ]
    
    # Определение категорий для выборки собственников бизнеса
    employment_segments_bus = [
        "Агро",
        "Производство",
        "Торговля",
        "Услуги",
        "Услуги Мастера",
        "Услуги транспорта",
        "Транспорт",
        "Производство строительных материалов",
        "Производство из железных материалов",
        "Растеневодство",
        "производство одежды",
        "Производство из стекла",
        "Садоводства",
        "Овощеводства",
        "Строительные материалы",
        "Пищевая промышленность",
        "Хоз. товары",
        "Косметика и парфюмерия",
        "Пчеловодство",
        "ГСМ",
        "Галантерея",
        "Сырьевые материалы",
        "Мебель",
        "Арендодатель",
        "Медикаменты",
        "Бытовая техника",
        "Золото",
        "Средства связи",
        "Финансовые услуги",
        "Переработка хлопка",
        "Производство из пластмассы",
        "производство бумаги",
        "Бахчевые",
        "менеральные удобрения",
        "рыбный промысел",
        "Производство из пиломатериалов"
    ]
    
    # Создание выборки для работников организаций
    sample_empl = df[
        (df['employment_type'] == "Работает в организации") &
        (df['employment_segment'].isin(employment_segments_empl))
    ].copy()
    
    # Создание выборки для собственников бизнеса
    sample_bus = df[
        (df['employment_type'] == "Собственный бизнес") &
        (df['employment_segment'].isin(employment_segments_bus))
    ].copy()
    
    # Создание выборки для других источников дохода (все остальные строки)
    sample_other = df[
        ~df.index.isin(sample_empl.index) &
        ~df.index.isin(sample_bus.index)
    ].copy()
    
    logging.info("Выборки по типу занятости созданы.")
    
    return sample_empl, sample_bus, sample_other

def log_and_verify_samples(df, sample_empl, sample_bus, sample_other):
    logging.info("Вывод размеров выборок.")
    print(f"\nРазмер выборки для работников: {sample_empl.shape[0]}")
    print(f"Размер выборки для бизнесменов: {sample_bus.shape[0]}")
    print(f"Размер выборки для других источников дохода: {sample_other.shape[0]}")
    
    # Проверка: совместный размер без пересечений
    combined_samples = pd.concat([sample_empl, sample_bus, sample_other]).drop_duplicates()
    print(f"\nОбщий размер выборок (без пересечений): {combined_samples.shape[0]}")
    print(f"Размер исходного DataFrame: {df.shape[0]}")
    
    if combined_samples.shape[0] != df.shape[0]:
        logging.warning("Сумма размеров выборок не соответствует общему размеру DataFrame. Проверьте пересечения и условия фильтрации.")
        # Дополнительная диагностика
        missing_count = df.shape[0] - combined_samples.shape[0]
        logging.warning(f"Количество отсутствующих строк: {missing_count}")
    else:
        logging.info("Сумма размеров выборок соответствует общему размеру DataFrame.")

def plot_employment_segment_distribution(df):
    logging.info("Построение графика распределения по 'employment_segment'.")
    
    plt.figure(figsize=(14, 12))
    sns.countplot(y='employment_segment', data=df, order=df['employment_segment'].value_counts().index)
    plt.title('Распределение по сегментам занятости')
    plt.xlabel('Частота')
    plt.ylabel('Сегмент занятости')
    plt.tight_layout()
    plt.show()
    
    logging.info("График распределения по 'employment_segment' построен.")


# Создание выборок
sample_empl, sample_bus, sample_other = create_employment_samples(df_selected)

# Логирование и проверка размеров выборок
log_and_verify_samples(df_selected, sample_empl, sample_bus, sample_other)

# Построение графика распределения
# plot_employment_segment_distribution(df_selected)



2024-12-12 13:02:33,255 - INFO - Создание выборок по типу занятости.
2024-12-12 13:02:33,319 - INFO - Выборки по типу занятости созданы.
2024-12-12 13:02:33,319 - INFO - Вывод размеров выборок.



Размер выборки для работников: 51539
Размер выборки для бизнесменов: 80114
Размер выборки для других источников дохода: 104033

Общий размер выборок (без пересечений): 235686
Размер исходного DataFrame: 235686


2024-12-12 13:02:33,506 - INFO - Сумма размеров выборок соответствует общему размеру DataFrame.


In [70]:
logging.info("Вывод размеров выборок.")
print(f"\nРазмер выборки для работников: {sample_empl.shape[0]}")
print(f"Размер выборки для бизнесменов: {sample_bus.shape[0]}")
print(f"Размер выборки для других источников дохода: {sample_other.shape[0]}")

2024-12-12 13:02:33,509 - INFO - Вывод размеров выборок.



Размер выборки для работников: 51539
Размер выборки для бизнесменов: 80114
Размер выборки для других источников дохода: 104033


In [71]:
df.head(5)

,customer_id,application_id,account_id,branch_id,product_id,application_date,date_loan_granted,loan_amount,first_instalment_due_date,interest_rate,collateral_type,value_of_collateral,property_type,salary_payment_in_bank_account,loan_type,number_of_instalments,instalment_amount,date_of_birth,gender,city_of_living,region_of_living,city_of_registration,region_of_registration,work_phone_number,mobile_phone_number,education,marital_status,dependants,number_children,months_at_current_address,employment_type,employment_sector,employment_segment,months_at_job,net_main_income,source_of_main_income,additional_income,source_of_additional_income,reported_expenses,months_with_bank,current_exposure,client_type,property_object,eskhata_online,plastic_cards,deposit,state,rejection_reason,run_date,date_account_opened,current_balance,date_last_payment,date_final_payment,due_date,payment_amount,account_status,payments_in_arrears,cumulative_delinquency,amount_due,principal_amount,interest_accrued,outstanding_balance,arrears_amount,current_dpd,max_dpd,max_dpd_lifetime,default_flag,number_of_extensions,bki_rating,bki_number_of_loans,ftd_1,ftd_2,ftd_3,ftd_4,age,loan_month,city_of_living_eq_registration,is_collateral,previous_loans_count,gb_90ever,gb_cum_dlq_90,gb_60ever
0,25121517058,764446/КР,35619143897,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ДУШАНБЕ",Карзхои гуногунмаксад,2021-10-19,2021-11-02,12300.0,2021-12-02,31.0,NaN,NaN,NaN,Нет,Многоцелевые кредиты_005_аннуитет,18,863.0,1998-01-07,Женский,нохияи Фирдавси,Душанбе,Фирдавси,Душанбе,992000805085,NaN,Высшее,Не замужем,0,2,0,Имеет другой источник дохода,NaN,NaN,0,3121.36,Прочее,NaN,NaN,1272.0,36,0.00,0,Квартира,Да,Да,0.0,NaN,NaN,2023-06-30,2021-11-02,0.0,NaT,2023-04-20,NaT,NaN,Закрыт,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,NaN,0,0,0,0,0,23,202110,False,False,0,0,0,0
1,847140141,766801/КР,35733163635,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ИСТАРАВШАН",Карзхои гуногунмаксад,2021-10-25,2021-11-01,20000.0,2021-12-01,30.0,1) Поручитель;,1) 0;,1) Поручитель;,Нет,Многоцелевые кредиты_005_аннуитет,24,1118.0,1969-05-16,Мужской,Истаравшан,Вилояти Сугд,Истаравшан,Вилояти Сугд,992985675558,NaN,Среднее,Женат,2,2,6,Собственный бизнес,Самозанятость,Услуги транспорта,276,3800.00,Прочее,NaN,NaN,1500.0,143,4691.17,1,Дом,Нет,Нет,0.0,NaN,NaN,2023-06-30,2021-11-01,0.0,2023-04-26,2023-10-26,2023-06-01,1118.0,Закрыт,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,NaN,5,0,0,0,0,52,202110,True,False,0,0,0,0
2,6286580057,766319/КР,35736913632,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ИСТАРАВШАН",Карзхои гуногунмаксад,2021-10-25,2021-11-05,10000.0,2021-12-06,30.0,NaN,NaN,NaN,Нет,Многоцелевые кредиты_005_аннуитет,24,559.0,1991-03-02,Мужской,Истаравшан,Вилояти Сугд,Истаравшан,Вилояти Сугд,992988621755,NaN,Среднее,Женат,3,2,9,Собственный бизнес,Самозанятость,Услуги транспорта,60,3500.00,Прочее,NaN,NaN,1000.0,61,4301.47,1,Дом,Нет,Нет,0.0,NaN,NaN,2023-06-30,2021-11-05,0.0,2023-04-03,2023-10-26,2023-06-05,559.0,Закрыт,NaN,19,0.0,0.0,0.0,0.0,0.0,0,0,7,NaN,0,NaN,2,0,1,0,7,30,202110,True,False,0,0,0,0
3,14939826396,766446/КР,35741587858,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш. КУЛОБ",Карзхои гуногунмаксад,2021-10-25,2021-11-03,3300.0,2021-12-03,31.0,NaN,NaN,NaN,Нет,Многоцелевые кредиты_005_аннуитет,12,323.0,1966-10-23,Мужской,Фархор,Вилояти Хатлон,Пархар,Вилояти Хатлон,+992900078418; ; ;,NaN,Среднее,Женат,1,2,39,Имеет другой источник дохода,NaN,NaN,0,2400.00,Доход семьи,NaN,NaN,1000.0,60,1153.52,1,Дом,Нет,Нет,0.0,NaN,NaN,2023-06-30,2021-11-03,0.0,2022-09-12,2022-10-26,NaT,NaN,Закрыт,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,NaN,0,0,0,0,0,55,202110,False,False,0,0,0,0
4,32830136711,767392/КР,35754732217,"ФИЛИАЛИ ЧСК ""БОНКИ ЭСХАТА"" ДАР Ш.ДУШАНБЕ, Н.СИ...",Карзхои гуногунмаксад,2021-10-26,2021-11-03,5000.0,2021-12-03,31.0,NaN,NaN,NaN,Нет,Многоцелевые кредиты_005_аннуитет,6,910.0,2001-06-07,Мужской,поселки Варзобского района,Нохияхои тобеи Чумхури,поселки Варзобского района,Нохияхои тобеи Чумхури,NaN,992888051515,Среднее,Холост,0,0,2,Собственный бизнес,Самозанятость,Ус

# Создание выборки для анализа

In [72]:
# Пример выбора выборки для анализа
# sample = sample_empl.copy()
# sample = sample_bus.copy()
sample = sample_other.copy()

logging.info("Структура выбранной выборки:")
print(sample.info())

2024-12-12 13:02:33,543 - INFO - Структура выбранной выборки:


<class 'pandas.core.frame.DataFrame'>
Index: 104033 entries, 0 to 235697
Data columns (total 44 columns):
 #   Column                          Non-Null Count   Dtype   
---  ------                          --------------   -----   
 0   loan_amount                     104033 non-null  float64 
 1   is_collateral                   104033 non-null  category
 2   salary_payment_in_bank_account  104033 non-null  category
 3   age                             104033 non-null  int64   
 4   gender                          104033 non-null  category
 5   region_of_registration          104033 non-null  category
 6   city_of_living_eq_registration  104033 non-null  category
 7   education                       104033 non-null  category
 8   marital_status                  104033 non-null  category
 9   dependants                      104033 non-null  int64   
 10  months_at_current_address       104033 non-null  int64   
 11  employment_type                 104033 non-null  category
 12  employm

# Определение целевой переменной

In [73]:
# Создание переменной gb
logging.info("Создание целевой переменной 'gb'.")
sample.loc[:, 'gb'] = sample['gb_60ever']
sample.gb = sample.gb.astype(int)

# sample.loc[:, 'gb'] = sample['gb_cum_dlq_90']

2024-12-12 13:02:33,559 - INFO - Создание целевой переменной 'gb'.


# Семплирование
## Создание выборок для Train и Test

In [74]:
# Выборка для обучения модели
dev = sample[(sample['loan_month'] >= 202110) & (sample['loan_month'] < 202210)].drop(columns=['loan_month']).copy()

# Выборка для тестирования модели на временную устойчивость
oot = sample[(sample['loan_month'] >= 202210) & (sample['loan_month'] < 202303)].copy()

# Выборка для кросс-валидации внутри обучающей выборки
oot2 = sample[(sample['loan_month'] >= 202207) & (sample['loan_month'] < 202210)].copy()

# Выборка для финального обучения модели
dev_final = sample[(sample['loan_month'] >= 202110) & (sample['loan_month'] < 202302)].drop(columns=['loan_month']).copy()

In [75]:
# Изучение распределений с графиками
logging.info("Изучение распределений в выборках.")

# Функция для подсчета распределений и построения графика
def plot_distribution(sample, sample_name):
    print(f"\nРаспределение в {sample_name}:")
    
    # Подсчитываем количество каждого значения в 'gb' (0 и 1)
    distribution = sample['gb'].value_counts()
    print(distribution)
    
    # Рассчитываем доли значений
    proportion_1 = distribution.get(1, 0) / len(sample)
    proportion_0 = distribution.get(0, 0) / len(sample)
    
    print(f"Доля значения '1' в {sample_name}: {proportion_1:.4f}")
    print(f"Доля значения '0' в {sample_name}: {proportion_0:.4f}")
    
#     # Построение графика распределения
#     plt.figure(figsize=(6, 4))
#     distribution.plot(kind='bar', color=['skyblue', 'lightcoral'])
#     plt.title(f"Распределение значений в {sample_name}")
#     plt.xlabel('Значение gb')
#     plt.ylabel('Количество')
#     plt.xticks(rotation=0)
#     plt.tight_layout()
#     plt.show()

# Распределение для каждой выборки с графиками
plot_distribution(dev, "Development Sample")
plot_distribution(oot, "Out-of-time Sample")
plot_distribution(oot2, "Out-of-time 2 Sample")
plot_distribution(dev_final, "Development Final Sample")


2024-12-12 13:02:33,604 - INFO - Изучение распределений в выборках.



Распределение в Development Sample:
gb
0    56815
1     1185
Name: count, dtype: int64
Доля значения '1' в Development Sample: 0.0204
Доля значения '0' в Development Sample: 0.9796

Распределение в Out-of-time Sample:
gb
0    25567
1      114
Name: count, dtype: int64
Доля значения '1' в Out-of-time Sample: 0.0044
Доля значения '0' в Out-of-time Sample: 0.9956

Распределение в Out-of-time 2 Sample:
gb
0    16525
1      177
Name: count, dtype: int64
Доля значения '1' в Out-of-time 2 Sample: 0.0106
Доля значения '0' в Out-of-time 2 Sample: 0.9894

Распределение в Development Final Sample:
gb
0    76654
1     1288
Name: count, dtype: int64
Доля значения '1' в Development Final Sample: 0.0165
Доля значения '0' в Development Final Sample: 0.9835


In [76]:
# Вывод количества строк в каждой выборке
logging.info("Вывод количества строк в каждой выборке.")
print(f"\nКоличество строк в Development Sample: {dev.shape[0]}")
print(f"Количество строк в Out-of-time Sample: {oot.shape[0]}")
print(f"Количество строк в Out-of-time 2 Sample: {oot2.shape[0]}")
print(f"Количество строк в Development Final Sample: {dev_final.shape[0]}")


2024-12-12 13:02:33,609 - INFO - Вывод количества строк в каждой выборке.



Количество строк в Development Sample: 58000
Количество строк в Out-of-time Sample: 25681
Количество строк в Out-of-time 2 Sample: 16702
Количество строк в Development Final Sample: 77942


# MACHINE LEARNING

In [77]:
import pandas as pd
import numpy as np
import json
import scorecardpy as sc
import statsmodels.api as sm
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (make_scorer, precision_score, recall_score, accuracy_score, roc_curve,
                             f1_score, auc, roc_auc_score, confusion_matrix, classification_report)
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE






# Расчет Information Value (IV)

In [78]:
logging.info("Расчет Information Value (IV) для всех переменных в dev_final.")
iv_values_final = sc.iv(dt=dev_final, y='gb')


2024-12-12 13:02:34,356 - INFO - Расчет Information Value (IV) для всех переменных в dev_final.


In [79]:
# Просмотр структуры и первых строк DataFrame iv_values_final
print("Структура iv_values_final:")
print(iv_values_final.columns)

print("\nПервые 5 строк iv_values_final:")
print(iv_values_final.head())


Структура iv_values_final:
Index(['variable', 'info_value'], dtype='object')

Первые 5 строк iv_values_final:
             variable  info_value
37          gb_60ever   18.605405
16      gb_cum_dlq_90    8.207865
6           gb_90ever    6.836621
30        loan_amount    1.655727
5   reported_expenses    1.630991


# Отбор переменных на основе IV

In [80]:
logging.info("Отбор переменных на основе IV >= 0.01.")
selected_vars_iv = iv_values_final[
    iv_values_final['info_value'] >= 0.01
]['variable'].tolist()

logging.info(f"Переменные с info_value >= 0.01: {selected_vars_iv}")


2024-12-12 13:02:40,348 - INFO - Отбор переменных на основе IV >= 0.01.
2024-12-12 13:02:40,349 - INFO - Переменные с info_value >= 0.01: ['gb_60ever', 'gb_cum_dlq_90', 'gb_90ever', 'loan_amount', 'reported_expenses', 'net_main_income', 'months_with_bank', 'months_at_job', 'bki_number_of_loans', 'age', 'loan_month_bin', 'age_bin', 'marital_status', 'months_with_bank_bin', 'bki_rating', 'region_of_registration', 'months_at_current_address', 'source_of_main_income', 'client_type', 'employment_segment', 'dependants', 'eskhata_online', 'months_at_current_address_bin', 'dependants_bin', 'employment_type', 'property_object', 'bki_number_of_loans_bin', 'gender', 'city_of_living_eq_registration', 'loan_amount_bin', 'plastic_cards', 'previous_loans_count', 'net_main_income_bin', 'education', 'months_at_job_bin', 'salary_payment_in_bank_account']


# Вычисление доли пропусков для отобранных переменных

In [81]:
logging.info("Расчет доли пропусков для отобранных переменных.")
# Рассчитываем долю пропусков для отобранных переменных
missing_rates = dev_final[selected_vars_iv].isnull().mean().reset_index()
missing_rates.columns = ['variable', 'missing']


2024-12-12 13:02:40,350 - INFO - Расчет доли пропусков для отобранных переменных.


In [82]:
# Просмотр первых строк
logging.info("Структура missing_rates:")
display(missing_rates.columns)

logging.info("missing_rates:")
display(missing_rates.sort_values(by='missing', ascending=False))


2024-12-12 13:02:40,357 - INFO - Структура missing_rates:


Index(['variable', 'missing'], dtype='object')

2024-12-12 13:02:40,358 - INFO - missing_rates:


,variable,missing
0,gb_60ever,0.0
1,gb_cum_dlq_90,0.0
20,dependants,0.0
21,eskhata_online,0.0
22,months_at_current_address_bin,0.0
23,dependants_bin,0.0
24,employment_type,0.0
25,property_object,0.0
26,bki_number_of_loans_bin,0.0
27,gender,0.0


# Объединение IV и missing_rates

In [83]:
# Объединяем IV и по долю пропусков
iv_with_missing = iv_values_final.merge(missing_rates, on='variable')

# Просмотр первых строк
logging.info("Структура iv_with_missing:")
display(iv_with_missing.columns)

logging.info("iv_with_missing:")
display(iv_with_missing)


2024-12-12 13:02:40,365 - INFO - Структура iv_with_missing:


Index(['variable', 'info_value', 'missing'], dtype='object')

2024-12-12 13:02:40,366 - INFO - iv_with_missing:


,variable,info_value,missing
0,gb_60ever,18.605405,0.0
1,gb_cum_dlq_90,8.207865,0.0
2,gb_90ever,6.836621,0.0
3,loan_amount,1.655727,0.0
4,reported_expenses,1.630991,0.0
5,net_main_income,1.540519,0.0
6,months_with_bank,0.493884,0.0
7,months_at_job,0.435982,0.0
8,bki_number_of_loans,0.425737,0.0
9,age,0.405788,0.0


# Фильтрация переменных по IV и пропускам

In [84]:
logging.info("Фильтрация переменных по info_value >= 0.01 и missing <= 0.95.")
# Отбор переменных с info_value >= 0.01 и пропусками <= 0.95
filtered_iv = iv_with_missing[
    (iv_with_missing['info_value'] >= 0.01) &
    (iv_with_missing['missing'] <= 0.95)
]

# Извлечение списка переменных
selected_vars_final = filtered_iv['variable'].tolist()

# Исключение определенных переменных
vars_to_remove = ["gb_90ever", "gb_60ever", "gb_cum_dlq_90", "current_exposure"]
selected_vars_final = [var for var in selected_vars_final if var not in vars_to_remove]

logging.info(f"Отобранные переменные после фильтрации по IV и пропускам: {selected_vars_final}")


2024-12-12 13:02:40,372 - INFO - Фильтрация переменных по info_value >= 0.01 и missing <= 0.95.
2024-12-12 13:02:40,373 - INFO - Отобранные переменные после фильтрации по IV и пропускам: ['loan_amount', 'reported_expenses', 'net_main_income', 'months_with_bank', 'months_at_job', 'bki_number_of_loans', 'age', 'loan_month_bin', 'age_bin', 'marital_status', 'months_with_bank_bin', 'bki_rating', 'region_of_registration', 'months_at_current_address', 'source_of_main_income', 'client_type', 'employment_segment', 'dependants', 'eskhata_online', 'months_at_current_address_bin', 'dependants_bin', 'employment_type', 'property_object', 'bki_number_of_loans_bin', 'gender', 'city_of_living_eq_registration', 'loan_amount_bin', 'plastic_cards', 'previous_loans_count', 'net_main_income_bin', 'education', 'months_at_job_bin', 'salary_payment_in_bank_account']


# Создание dev_final_sel

In [85]:
logging.info("Создание dev_final_sel с отобранными переменными.")
dev_final_sel = dev_final[selected_vars_final + ['gb']].copy()

logging.info(f"Количество строк в dev_final_sel: {dev_final_sel.shape[0]}")
logging.info(f"Количество столбцов в dev_final_sel: {dev_final_sel.shape[1]}")


2024-12-12 13:02:40,374 - INFO - Создание dev_final_sel с отобранными переменными.
2024-12-12 13:02:40,377 - INFO - Количество строк в dev_final_sel: 77942
2024-12-12 13:02:40,378 - INFO - Количество столбцов в dev_final_sel: 34


# Проверка пропусков

In [86]:
# Проверка пропущенных значений
print("Количество пропущенных значений в каждом столбце:")
print(dev_final_sel.isnull().sum())

Количество пропущенных значений в каждом столбце:
loan_amount                       0
reported_expenses                 0
net_main_income                   0
months_with_bank                  0
months_at_job                     0
bki_number_of_loans               0
age                               0
loan_month_bin                    0
age_bin                           0
marital_status                    0
months_with_bank_bin              0
bki_rating                        0
region_of_registration            0
months_at_current_address         0
source_of_main_income             0
client_type                       0
employment_segment                0
dependants                        0
eskhata_online                    0
months_at_current_address_bin     0
dependants_bin                    0
employment_type                   0
property_object                   0
bki_number_of_loans_bin           0
gender                            0
city_of_living_eq_registration    0
loan_amount_bi

# Определение правил биннинга

In [87]:
logging.info("Определение правил биннинга для переменных.")

breaks = {
    'education': [
        "Высшее", "Ученая степень", "2 и более высших",
        "Среднее", "Начальное",
        "Среднее специальное",
        "Неоконченное высшее",
        "missing"
    ],
    'region_of_registration': [
        "Вилояти Сугд", 
        "Вилояти Хатлон", "ВМКБ",
        "Душанбе", 
        "Нохияхои тобеи Чумхури"
    ],
    'marital_status': [
        "Женат", "Сожитель",
        "Замужем", "Вдова", "разведена",
        "Холост", 
        "Не замужем", "Разведен"
    ],
    'source_of_main_income': [
        "Доход семьи",
        "Зарплата по основному месту работы",
        "Предпринимательство",
        "Пенсия", "Алименты", "Прочее", "missing"
    ]
}


2024-12-12 13:02:40,383 - INFO - Определение правил биннинга для переменных.


In [88]:
logging.info("Начало подготовки данных для биннинга.")

# Определение исходных переменных (без бинированных столбцов и целевой переменной)
original_cols = [col for col in dev_final_sel.columns if not col.endswith('_bin') and col != 'gb']

# Создание DataFrame для биннинга
df_original = dev_final_sel[original_cols + ['gb']].copy()

2024-12-12 13:02:40,384 - INFO - Начало подготовки данных для биннинга.


In [94]:
# Запуск биннинга
logging.info("Запуск биннинга и расчета WOE.")

try:
    bins_fin = sc.woebin(
        dt=df_original,
        y="gb",
        method="tree",
        breaks_list=breaks,
        check_cate_num=True,
        count_distr_limit=0.05,
        bin_num_limit=5
    )
    logging.info("Биннинг и расчет WOE выполнены успешно.")
except Exception as e:
    logging.error(f"Ошибка при биннинге и расчете WOE: {e}")
    raise

2024-12-12 13:03:30,705 - INFO - Запуск биннинга и расчета WOE.


[INFO] creating woe binning ...


2024-12-12 13:03:36,940 - INFO - Биннинг и расчет WOE выполнены успешно.


In [90]:
# Применение правил биннинга для получения WOE переменных
logging.info("Применение правил биннинга для получения WOE переменных.")

try:
    df_woe = sc.woebin_ply(
        dt=df_original,
        bins=bins_fin,
        print_step=0  # Отключение подробного вывода
    )
    logging.info("WOE переменные успешно добавлены к данным.")
except Exception as e:
    logging.error(f"Ошибка при применении WOE: {e}")
    raise

# Объединение исходного датасета с WOE-кодированными переменными
logging.info("Объединение исходного датасета с WOE-кодированными переменными.")

# Определение WOE-переменных
woe_cols = [col for col in df_woe.columns if 'woe' in col]

# Создание финального датасета с исходными данными и WOE-переменными
final_bins = df_original.join(df_woe[woe_cols], how='left')

logging.info("Объединение данных завершено.")


2024-12-12 13:02:46,374 - INFO - Применение правил биннинга для получения WOE переменных.


[INFO] converting into woe values ...


2024-12-12 13:02:50,111 - INFO - WOE переменные успешно добавлены к данным.
2024-12-12 13:02:50,111 - INFO - Объединение исходного датасета с WOE-кодированными переменными.
2024-12-12 13:02:50,141 - INFO - Объединение данных завершено.


In [91]:
logging.info('Запись результата бининга в excel')
# with pd.ExcelWriter('output.xlsx') as writer: 
#     for k, v in final_bins.items(): 
#         v.to_excel(writer, sheet_name=k)


2024-12-12 13:02:50,142 - INFO - Запись результата бининга в excel


# Разбиение данных на Train и Test

In [92]:
logging.info("Разбиение данных на Train и Test (Out-of-sample).")
train, test = train_test_split(
    dev_final_sel,
    test_size=0.3,
    random_state=42,
    stratify=dev_final_sel['gb']
)

logging.info(f"Количество строк в Train: {train.shape[0]}")
logging.info(f"Количество строк в Test: {test.shape[0]}")


2024-12-12 13:02:50,142 - INFO - Разбиение данных на Train и Test (Out-of-sample).
2024-12-12 13:02:50,170 - INFO - Количество строк в Train: 54559
2024-12-12 13:02:50,171 - INFO - Количество строк в Test: 23383


In [96]:
dev_final_sel.columns

Index(['loan_amount', 'reported_expenses', 'net_main_income',
       'months_with_bank', 'months_at_job', 'bki_number_of_loans', 'age',
       'loan_month_bin', 'age_bin', 'marital_status', 'months_with_bank_bin',
       'bki_rating', 'region_of_registration', 'months_at_current_address',
       'source_of_main_income', 'client_type', 'employment_segment',
       'dependants', 'eskhata_online', 'months_at_current_address_bin',
       'dependants_bin', 'employment_type', 'property_object',
       'bki_number_of_loans_bin', 'gender', 'city_of_living_eq_registration',
       'loan_amount_bin', 'plastic_cards', 'previous_loans_count',
       'net_main_income_bin', 'education', 'months_at_job_bin',
       'salary_payment_in_bank_account', 'gb'],
      dtype='object')

In [99]:
logging.debug(f"Структура bins_fin: {bins_fin.keys()}")

# Применение WOE кодирования к всем выборкам

In [100]:
logging.info("Применение WOE кодирования ко всем выборкам.")

# Список выборок для кодирования
datasets = {
    'dev': dev,
    'oot': oot,
    'oot2': oot2,
    'train': train,
    'test': test,
    'dev_final_sel': dev_final_sel,
    'sample': sample
}

# Словарь для хранения закодированных выборок
encoded_datasets = {}

for name, dataset in datasets.items():
    try:
        logging.info(f"Начало WOE кодирования для выборки '{name}'.")
        encoded = sc.woebin_ply(dataset, bins_fin, print_step=0)  # Используем bins_fin, а не final_bins
        encoded_datasets[f"{name}_woe"] = encoded
        logging.info(f"WOE кодирование для выборки '{name}' выполнено успешно.")
    except Exception as e:
        logging.error(f"Ошибка при WOE кодировании выборки '{name}': {e}")
        raise

# Извлечение закодированных выборок из словаря
dev_woe = encoded_datasets['dev_woe']
oot_woe = encoded_datasets['oot_woe']
oot2_woe = encoded_datasets['oot2_woe']
train_woe = encoded_datasets['train_woe']
test_woe = encoded_datasets['test_woe']
dev_final_woe = encoded_datasets['dev_final_sel_woe']
sample_woe = encoded_datasets['sample_woe']

logging.info("WOE кодирование всех выборок завершено успешно.")


2024-12-12 13:07:29,973 - INFO - Применение WOE кодирования ко всем выборкам.
2024-12-12 13:07:29,973 - INFO - Начало WOE кодирования для выборки 'dev'.


[INFO] converting into woe values ...


2024-12-12 13:07:34,708 - INFO - WOE кодирование для выборки 'dev' выполнено успешно.
2024-12-12 13:07:34,708 - INFO - Начало WOE кодирования для выборки 'oot'.


[INFO] converting into woe values ...


2024-12-12 13:07:37,089 - INFO - WOE кодирование для выборки 'oot' выполнено успешно.
2024-12-12 13:07:37,089 - INFO - Начало WOE кодирования для выборки 'oot2'.


...

2024-12-12 13:07:44,097 - INFO - WOE кодирование для выборки 'test' выполнено успешно.
2024-12-12 13:07:44,098 - INFO - Начало WOE кодирования для выборки 'dev_final_sel'.


[INFO] converting into woe values ...


2024-12-12 13:07:49,214 - INFO - WOE кодирование для выборки 'dev_final_sel' выполнено успешно.
2024-12-12 13:07:49,214 - INFO - Начало WOE кодирования для выборки 'sample'.


[INFO] converting into woe values ...


2024-12-12 13:07:57,551 - INFO - WOE кодирование для выборки 'sample' выполнено успешно.
2024-12-12 13:07:57,552 - INFO - WOE кодирование всех выборок завершено успешно.


# Расчет Information Value для WOE-переменных

In [101]:
logging.info("Расчет Information Value для WOE-переменных.")
iv_values_final_woe = sc.iv(dt=dev_final_woe, y='gb')

# Просмотр структуры и первых строк IV для WOE-переменных
print("Структура iv_values_final_woe:")
print(iv_values_final_woe.columns)

print("\niv_values_final_woe:")
print(iv_values_final_woe)

# Сохранение IV значений
iv_values_final_woe.to_excel("iv_values_woe.xlsx", index=False)

2024-12-12 13:07:57,553 - INFO - Расчет Information Value для WOE-переменных.


Структура iv_values_final_woe:
Index(['variable', 'info_value'], dtype='object')

iv_values_final_woe:
                              variable  info_value
7                       loan_month_bin    0.390844
28                             age_woe    0.344255
23                             age_bin    0.312737
17                  marital_status_woe    0.304767
22                months_with_bank_woe    0.256128
14                months_with_bank_bin    0.249313
25          region_of_registration_woe    0.201345
27           source_of_main_income_woe    0.183017
2                       bki_rating_woe    0.179368
11                     client_type_woe    0.138413
9        months_at_current_address_woe    0.130581
5              bki_number_of_loans_woe    0.130274
16                  eskhata_online_woe    0.118043
19       months_at_current_address_bin    0.110262
6                       dependants_bin    0.106165
32                   months_at_job_woe    0.106132
30                      depend

# Проверка корреляции между признаками

In [107]:
# Создаем список переменных
selected_vars_final = iv_values_final_woe['variable'].tolist()
selected_vars_final


['loan_month_bin', 'age_woe', 'age_bin', 'marital_status_woe', 'months_with_bank_woe', 'months_with_bank_bin', 'region_of_registration_woe', 'source_of_main_income_woe', 'bki_rating_woe', 'client_type_woe', 'months_at_current_address_woe', 'bki_number_of_loans_woe', 'eskhata_online_woe', 'months_at_current_address_bin', 'dependants_bin', 'months_at_job_woe', 'dependants_woe', 'employment_type_woe', 'bki_number_of_loans_bin', 'property_object_woe', 'loan_amount_woe', 'gender_woe', 'employment_segment_woe', 'city_of_living_eq_registration_woe', 'loan_amount_bin', 'plastic_cards_woe', 'net_main_income_woe', 'previous_loans_count_woe', 'net_main_income_bin', 'education_woe', 'months_at_job_bin', 'reported_expenses_woe', 'salary_payment_in_bank_account_woe']

In [108]:
# Функция для преобразования интервалов в числовые значения
def interval_to_numeric(interval):
    if pd.isnull(interval):
        return np.nan
    return (interval.left + interval.right) / 2

# Преобразуем бинированные переменные в числовой формат
for var in selected_vars_final:
    if var.endswith('_bin'):
        try:
            dev_final_woe[var] = dev_final_woe[var].apply(interval_to_numeric)
            logging.info(f"Преобразование переменной '{var}' в числовой формат выполнено успешно.")
        except Exception as e:
            logging.error(f"Ошибка при преобразовании переменной '{var}': {e}")
            raise



2024-12-12 13:10:44,914 - ERROR - Ошибка при преобразовании переменной 'loan_month_bin': 'float' object has no attribute 'left'


AttributeError: 'float' object has no attribute 'left'

In [109]:
logging.info("Расчет корреляционной матрицы.")
try:
    cor_matrix = dev_final_woe[selected_vars_final].corr()
    cor_matrix.to_excel("employee_corr_matrix.xlsx")
    logging.info("Корреляционная матрица сохранена в 'employee_corr_matrix.xlsx'.")
except Exception as e:
    logging.error(f"Ошибка при расчете корреляционной матрицы: {e}")
    raise

logging.info("Визуализация корреляционной матрицы.")
try:
    plt.figure(figsize=(12, 10))
    sns.heatmap(cor_matrix, annot=True, fmt=".2f", cmap='coolwarm')
    plt.title('Корреляционная матрица')
    plt.tight_layout()
    plt.show()
    logging.info("Визуализация корреляционной матрицы выполнена успешно.")
except Exception as e:
    logging.error(f"Ошибка при визуализации корреляционной матрицы: {e}")
    raise


2024-12-12 13:10:45,623 - INFO - Расчет корреляционной матрицы.
2024-12-12 13:10:45,903 - INFO - Корреляционная матрица сохранена в 'employee_corr_matrix.xlsx'.
2024-12-12 13:10:45,903 - INFO - Визуализация корреляционной матрицы.
2024-12-12 13:10:56,475 - INFO - Визуализация корреляционной матрицы выполнена успешно.


# Пошаговый отбор переменных (Stepwise Selection)

In [110]:
# Функция для пошагового отбора переменных на основе p-значений
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out=0.05, 
                       verbose=True):
    """Пошаговый отбор переменных на основе p-значений."""
    included = list(initial_list)
    while True:
        changed=False
        # Добавление переменной
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_column in excluded:
            try:
                model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit(disp=0)
                new_pval[new_column] = model.pvalues[new_column]
            except:
                new_pval[new_column] = 1
        if not new_pval.empty:
            best_pval = new_pval.min()
            if best_pval < threshold_in:
                best_feature = new_pval.idxmin()
                included.append(best_feature)
                changed=True
                if verbose:
                    print(f'Добавлена переменная {best_feature} с p-value {best_pval}')
        # Удаление переменной
        if len(included) > 0:
            model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included]))).fit(disp=0)
            pvalues = model.pvalues.drop('const')
            worst_pval = pvalues.max()
            if worst_pval > threshold_out:
                changed=True
                worst_feature = pvalues.idxmax()
                included.remove(worst_feature)
                if verbose:
                    print(f'Удалена переменная {worst_feature} с p-value {worst_pval}')
        if not changed:
            break
    return included


# Разделение данных на признаки и целевую переменную

In [111]:
logging.info("Разделение данных на признаки и целевую переменную.")
X_train = train_woe.drop(['gb'], axis=1)
y_train = train_woe['gb']

# Пошаговый отбор переменных на основе p-значений
logging.info("Пошаговый отбор переменных на основе p-значений.")
selected_features_stepwise = stepwise_selection(X_train, y_train)

logging.info(f"Отобранные переменные после пошагового отбора: {selected_features_stepwise}")

2024-12-12 13:11:09,815 - INFO - Разделение данных на признаки и целевую переменную.
2024-12-12 13:11:09,822 - INFO - Пошаговый отбор переменных на основе p-значений.


Добавлена переменная marital_status_woe с p-value 1.130505753405644e-68
Добавлена переменная months_with_bank_woe с p-value 1.17916591523488e-37
Добавлена переменная region_of_registration_woe с p-value 1.139903872917655e-23
Добавлена переменная bki_rating_woe с p-value 1.9211109715873883e-22
Добавлена переменная age_woe с p-value 1.994630968579694e-19
Добавлена переменная source_of_main_income_woe с p-value 2.7771783792648527e-16
Добавлена переменная loan_amount_woe с p-value 3.339561932833822e-15
Добавлена переменная dependants_woe с p-value 3.1025960965187993e-07
Добавлена переменная eskhata_online_woe с p-value 3.927487945413464e-06
Добавлена переменная months_at_current_address_woe с p-value 0.0004304757072928373
Добавлена переменная gender_woe с p-value 0.0008497330979875362
Добавлена переменная property_object_woe с p-value 0.0006126313490688216


2024-12-12 13:11:40,143 - INFO - Отобранные переменные после пошагового отбора: ['marital_status_woe', 'months_with_bank_woe', 'region_of_registration_woe', 'bki_rating_woe', 'age_woe', 'source_of_main_income_woe', 'loan_amount_woe', 'dependants_woe', 'eskhata_online_woe', 'months_at_current_address_woe', 'gender_woe', 'property_object_woe']


# Настройка модели с отобранными переменными

In [126]:
logging.info("Обучение модели с отобранными переменными.")

# Отобранные признаки
X_train_selected = X_train[selected_features_stepwise]

best_model = LogisticRegression(max_iter=1000, class_weight='balanced')

# Определение сетки гиперпараметров
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Параметр регуляризации
    'penalty': ['l2'],  # Тип регуляризации (L2 для логистической регрессии)
    'solver': ['lbfgs', 'liblinear'],  # Алгоритм оптимизации
    'max_iter': [1000, 1500, 2000],  # Максимальное количество итераций
}

# Инициализация GridSearchCV с использованием кросс-валидации
grid_search = GridSearchCV(estimator=best_model,
                           param_grid=param_grid,
                           scoring=make_scorer(accuracy_score),  # Оценка по точности
                           cv=10,  # Количество фолдов для кросс-валидации
                           verbose=1,  # Логирование прогресса
                           n_jobs=-1)  # Использование всех доступных ядер процессора

# Поиск по сетке гиперпараметров
grid_search.fit(X_train_selected, y_train)

# Лучшие параметры и результат
logging.info(f"Лучшие параметры: {grid_search.best_params_}")
logging.info(f"Лучший результат: {grid_search.best_score_:.4f}")


2024-12-12 13:18:00,074 - INFO - Обучение модели с отобранными переменными.
2024-12-12 13:18:00,074 - INFO - Подготовка данных для обучения и тестирования.


[INFO] converting into woe values ...
[INFO] converting into woe values ...


2024-12-12 13:18:05,950 - INFO - Начало обучения модели с GridSearchCV.


Fitting 5 folds for each of 12 candidates, totalling 60 fits


2024-12-12 13:18:10,390 - INFO - Лучшие параметры: {'model__C': 0.001, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
2024-12-12 13:18:10,391 - INFO - Лучший результат (F1): 0.0746
2024-12-12 13:18:10,391 - INFO - Оценка модели на тестовой выборке.
2024-12-12 13:18:10,469 - INFO - ROC-AUC на тесте: 0.7859
2024-12-12 13:18:10,470 - INFO - F1-Score на тесте: 0.0778
2024-12-12 13:18:10,470 - INFO - Classification Report на тесте:
2024-12-12 13:18:10,527 - INFO - 
              precision    recall  f1-score   support

           0       0.99      0.73      0.84     22997
           1       0.04      0.70      0.08       386

    accuracy                           0.73     23383
   macro avg       0.52      0.71      0.46     23383
weighted avg       0.98      0.73      0.83     23383

2024-12-12 13:18:10,532 - INFO - Confusion Matrix на тесте:
[[16683  6314]
 [  115   271]]


...

2024-12-12 13:18:22,123 - INFO - OOT - ROC-AUC: 0.7554, F1-Score: 0.0223
2024-12-12 13:18:22,128 - INFO - OOT - Confusion Matrix:
[[20272  5295]
 [   53    61]]


[INFO] converting into woe values ...


2024-12-12 13:18:27,270 - INFO - OOT2 - ROC-AUC: 0.7680, F1-Score: 0.0467
2024-12-12 13:18:27,274 - INFO - OOT2 - Confusion Matrix:
[[11850  4675]
 [   61   116]]


[INFO] converting into woe values ...


2024-12-12 13:18:36,401 - INFO - FULL - ROC-AUC: 0.7828, F1-Score: 0.0761
2024-12-12 13:18:36,415 - INFO - FULL - Confusion Matrix:
[[55214 21440]
 [  389   899]]
2024-12-12 13:18:39,538 - INFO - Обучение и оценка модели завершены успешно.


In [140]:
import matplotlib.pyplot as plt
import seaborn as sns

# Пример для OOT выборки
plt.figure(figsize=(8,6))
sns.histplot(y_scores, bins=50, kde=True)
plt.title('Распределение вероятностей предсказаний на OOT выборке')
plt.xlabel('Вероятность')
plt.ylabel('Частота')
plt.show()


In [141]:
from sklearn.ensemble import RandomForestClassifier

# Создание пайплайна с Random Forest и SMOTE
rf_pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('model', RandomForestClassifier(class_weight='balanced', random_state=42))
])

# Определение сетки гиперпараметров
rf_param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2],
    'model__bootstrap': [True, False]
}

# Инициализация GridSearchCV
rf_grid_search = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=rf_param_grid,
    scoring='f1',
    cv=cv_strategy,
    verbose=2,
    n_jobs=-1
)

# Обучение модели
logging.info("Начало обучения модели Random Forest с GridSearchCV.")
rf_grid_search.fit(X_train_selected, y_train_selected)
logging.info(f"Лучшие параметры Random Forest: {rf_grid_search.best_params_}")
logging.info(f"Лучший результат Random Forest (F1-Score): {rf_grid_search.best_score_:.4f}")

# Оценка модели Random Forest на тестовой выборке
best_rf_model = rf_grid_search.best_estimator_
y_pred_rf = best_rf_model.predict(X_test_selected)
y_scores_rf = best_rf_model.predict_proba(X_test_selected)[:,1]

# Вычисление метрик
roc_auc_rf = roc_auc_score(y_test_selected, y_scores_rf)
f1_rf = f1_score(y_test_selected, y_pred_rf)
precision_rf = precision_score(y_test_selected, y_pred_rf)
recall_rf = recall_score(y_test_selected, y_pred_rf)

logging.info(f"Random Forest - ROC-AUC на тесте: {roc_auc_rf:.4f}")
logging.info(f"Random Forest - F1-Score на тесте: {f1_rf:.4f}")
logging.info(f"Random Forest - Precision на тесте: {precision_rf:.4f}")
logging.info(f"Random Forest - Recall на тесте: {recall_rf:.4f}")

# Матрица ошибок
cm_rf = confusion_matrix(y_test_selected, y_pred_rf)
logging.info(f"Random Forest - Confusion Matrix на тесте:\n{cm_rf}")

# Визуализация матрицы ошибок
plt.figure(figsize=(6,4))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Purples', xticklabels=['Pred Negative', 'Pred Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix Random Forest на тестовой выборке')
plt.tight_layout()
plt.show()


2024-12-12 13:27:01,976 - INFO - Начало обучения модели Random Forest с GridSearchCV.


Fitting 5 folds for each of 48 candidates, totalling 240 fits


2024-12-12 13:35:17,332 - INFO - Лучшие параметры Random Forest: {'model__bootstrap': False, 'model__max_depth': 10, 'model__min_samples_leaf': 1, 'model__min_samples_split': 5, 'model__n_estimators': 200}
2024-12-12 13:35:17,332 - INFO - Лучший результат Random Forest (F1-Score): 0.0998
2024-12-12 13:35:17,935 - INFO - Random Forest - ROC-AUC на тесте: 0.7552
2024-12-12 13:35:17,935 - INFO - Random Forest - F1-Score на тесте: 0.1001
2024-12-12 13:35:17,935 - INFO - Random Forest - Precision на тесте: 0.0597
2024-12-12 13:35:17,935 - INFO - Random Forest - Recall на тесте: 0.3109
2024-12-12 13:35:17,937 - INFO - Random Forest - Confusion Matrix на тесте:
[[21106  1891]
 [  266   120]]


# Обучение модели с лучшими гиперпараметрами

In [137]:
logging.info("Обучение модели с отобранными переменными.")
# Получение лучшей модели
model_selected = grid_search.best_estimator_

# Обучение лучшей модели
model_selected.fit(X_train_selected, y_train)

# Прогнозирование
# y_pred = best_model.predict(X_test_selected)

# Сохранение лучших гиперпараметров в лог
logging.info(f"Лучшие гиперпараметры: {grid_search.best_params_}")

2024-12-12 13:25:13,888 - INFO - Обучение модели с отобранными переменными.
2024-12-12 13:25:14,141 - INFO - Лучшие гиперпараметры: {'model__C': 0.001, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}


# Получение коэффициентов логистической регрессии

In [138]:
# # coef_ содержит коэффициенты для каждого признака

# importances = np.round(model_selected.coef_[0], 2)

# # Создание DataFrame для вывода важности признаков
# df_importances = pd.DataFrame({
#     'Feature': X_train_selected.columns,
#     'Importance': importances
# })

# # Сохранение результатов в файл Excel
# df_importances.to_excel('feature_importances.xlsx', index=False)

# # Построение графика важности признаков
# plt.barh(X_train_selected.columns, importances)
# plt.xlabel('Feature Importance')
# plt.ylabel('Feature')
# plt.title('Feature Importances for Selected Features')
# plt.show()

AttributeError: 'Pipeline' object has no attribute 'coef_'

In [ ]:
# # Вывод коэффициентов модели
# logging.info("Коэффициенты модели с отобранными переменными:")
# coefficients_selected = pd.DataFrame({
#     'Variable': selected_features_stepwise,
#     'Coefficient': model_selected.coef_[0]
# })
# print(coefficients_selected)

# Подготовка данных для предсказания

In [142]:
# Обучение и сохранение scaler и imputer
logging.info("Обучение SimpleImputer и StandardScaler.")
# Обучение импьютера на тренировочных данных
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train_selected)

# Обучение скейлера на импутиванных данных
X_train_imputed = imputer.transform(X_train_selected)
scaler = StandardScaler()
scaler.fit(X_train_imputed)

# Сохранение scaler и imputer
logging.info("Сохранение scaler и imputer на диск.")

# joblib.dump(scaler, 'scaler_rf.joblib')
# joblib.dump(imputer, 'imputer_rf.joblib')

# joblib.dump(scaler, 'scaler_logres.joblib')
# joblib.dump(imputer, 'imputer_logres.joblib')

2024-12-12 13:36:48,989 - INFO - Обучение SimpleImputer и StandardScaler.
2024-12-12 13:36:48,998 - INFO - Сохранение scaler и imputer на диск.


['imputer_rf.joblib']

In [144]:
# Загрузка scaler и imputer
scaler = joblib.load('scaler_rf.joblib')
imputer = joblib.load('imputer_rf.joblib')

# Функция для подготовки данных перед предсказанием
def prepare_data(df, bins, selected_features, scaler, imputer):
    """
    Применяет WOE кодирование, обработку пропусков и масштабирование к данным.
    """
    # Применение WOE кодирования
    df_woe = sc.woebin_ply(df, bins)
    
    # Выбор отобранных признаков
    X = df_woe[selected_features]
    
    # Обработка пропусков
    X_imputed = imputer.transform(X)
    
    # Масштабирование
    X_scaled = scaler.transform(X_imputed)
    
    return X_scaled

# Применение функции подготовки данных к различным выборкам

In [145]:
logging.info("Подготовка данных для предсказания на различных подвыборках.")

# Список подвыборок и соответствующих целевых переменных
datasets = {
    'train': train,
    'test': test,
    'oot': oot,
    'oot2': oot2,
    'dev_final': dev_final,
    'sample': sample
}

# Словарь для хранения подготовленных данных
prepared_data = {}

for name, dataset in datasets.items():
    try:
        logging.info(f"Подготовка данных для подвыборки '{name}'.")
        
        # Подготовка признаков
        X_scaled = prepare_data(dataset, bins_fin, selected_features_stepwise, scaler, imputer)
        # Извлечение целевой переменной
        y = dataset['gb']
        
        # Сохранение в словарь
        prepared_data[f'X_{name}_scaled'] = X_scaled
        prepared_data[f'y_{name}'] = y
        
        logging.info(f"Данные для подвыборки '{name}' подготовлены успешно.")
    except Exception as e:
        logging.error(f"Ошибка при подготовке данных для подвыборки '{name}': {e}")
        raise

# Извлечение подготовленных данных из словаря для дальнейшего использования
X_train_selected_scaled = prepared_data['X_train_scaled']
y_train_selected = prepared_data['y_train']

X_test_selected_scaled = prepared_data['X_test_scaled']
y_test_selected = prepared_data['y_test']

X_oot_selected_scaled = prepared_data['X_oot_scaled']
y_oot_selected = prepared_data['y_oot']

X_oot2_selected_scaled = prepared_data['X_oot2_scaled']
y_oot2_selected = prepared_data['y_oot2']

X_dev_final_selected_scaled = prepared_data['X_dev_final_scaled']
y_dev_final_selected = prepared_data['y_dev_final']

# Если требуется подготовить дополнительные выборки, например, 'sample'
X_sample_selected_scaled = prepared_data.get('X_sample_scaled')
y_sample_selected = prepared_data.get('y_sample')

logging.info("Подготовка данных для всех подвыборок завершена успешно.")

2024-12-12 13:37:47,332 - INFO - Подготовка данных для предсказания на различных подвыборках.
2024-12-12 13:37:47,332 - INFO - Подготовка данных для подвыборки 'train'.


[INFO] converting into woe values ...


2024-12-12 13:37:49,889 - INFO - Данные для подвыборки 'train' подготовлены успешно.
2024-12-12 13:37:49,889 - INFO - Подготовка данных для подвыборки 'test'.


[INFO] converting into woe values ...


2024-12-12 13:37:51,061 - INFO - Данные для подвыборки 'test' подготовлены успешно.
2024-12-12 13:37:51,061 - INFO - Подготовка данных для подвыборки 'oot'.


...

2024-12-12 13:37:53,508 - INFO - Данные для подвыборки 'oot2' подготовлены успешно.
2024-12-12 13:37:53,508 - INFO - Подготовка данных для подвыборки 'dev_final'.


[INFO] converting into woe values ...


2024-12-12 13:37:57,967 - INFO - Данные для подвыборки 'dev_final' подготовлены успешно.
2024-12-12 13:37:57,967 - INFO - Подготовка данных для подвыборки 'sample'.


[INFO] converting into woe values ...


2024-12-12 13:38:03,706 - INFO - Данные для подвыборки 'sample' подготовлены успешно.
2024-12-12 13:38:03,706 - INFO - Подготовка данных для всех подвыборок завершена успешно.


# Предсказания и оценка модели

In [146]:
# Функция для оценки модели
def evaluate_model(y_true, y_pred, y_pred_proba, dataset_name):
    """
    Выводит ROC-AUC, отчет по классификации и матрицу ошибок.
    """
    auc = roc_auc_score(y_true, y_pred_proba)
    logging.info(f"ROC-AUC на {dataset_name} выборке: {auc:.4f}")
    print(f"\nОтчёт по классификации на {dataset_name} выборке:")
    print(classification_report(y_true, y_pred))
    print(f"Матрица ошибок на {dataset_name} выборке:")
    print(confusion_matrix(y_true, y_pred))
    return auc

In [147]:
# Предсказания на различных подвыборках
logging.info("Предсказания на тренировочной выборке.")
train_pred_proba = model_selected.predict_proba(X_train_selected_scaled)[:, 1]
train_pred = model_selected.predict(X_train_selected_scaled)

logging.info("Предсказания на тестовой выборке (out-of-sample).")
test_pred_proba = model_selected.predict_proba(X_test_selected_scaled)[:, 1]
test_pred = model_selected.predict(X_test_selected_scaled)

logging.info("Предсказания на out-of-time выборке.")
oot_pred_proba = model_selected.predict_proba(X_oot_selected_scaled)[:, 1]
oot_pred = model_selected.predict(X_oot_selected_scaled)

logging.info("Предсказания на out-of-time 2 выборке.")
oot2_pred_proba = model_selected.predict_proba(X_oot2_selected_scaled)[:, 1]
oot2_pred = model_selected.predict(X_oot2_selected_scaled)

logging.info("Предсказания на Development Final выборке.")
dev_final_pred_proba = model_selected.predict_proba(X_dev_final_selected_scaled)[:, 1]
dev_final_pred = model_selected.predict(X_dev_final_selected_scaled)


2024-12-12 13:38:03,708 - INFO - Предсказания на тренировочной выборке.
2024-12-12 13:38:03,731 - INFO - Предсказания на тестовой выборке (out-of-sample).
2024-12-12 13:38:03,732 - INFO - Предсказания на out-of-time выборке.
2024-12-12 13:38:03,742 - INFO - Предсказания на out-of-time 2 выборке.
2024-12-12 13:38:03,748 - INFO - Предсказания на Development Final выборке.


# Оценка модели на различных подвыборках

In [148]:
auc_train = evaluate_model(y_train_selected, train_pred, train_pred_proba, "тренировочной")
auc_test = evaluate_model(y_test_selected, test_pred, test_pred_proba, "тестовой (out-of-sample)")
auc_oot = evaluate_model(y_oot_selected, oot_pred, oot_pred_proba, "out-of-time")
auc_oot2 = evaluate_model(y_oot2_selected, oot2_pred, oot2_pred_proba, "out-of-time 2")
auc_dev_final = evaluate_model(y_dev_final_selected, dev_final_pred, dev_final_pred_proba, "Development Final")


2024-12-12 13:38:03,785 - INFO - ROC-AUC на тренировочной выборке: 0.7775



Отчёт по классификации на тренировочной выборке:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     53657
           1       0.08      0.26      0.12       902

    accuracy                           0.94     54559
   macro avg       0.53      0.61      0.55     54559
weighted avg       0.97      0.94      0.95     54559

Матрица ошибок на тренировочной выборке:
[[50948  2709]
 [  667   235]]


2024-12-12 13:38:03,862 - INFO - ROC-AUC на тестовой (out-of-sample) выборке: 0.7820



Отчёт по классификации на тестовой (out-of-sample) выборке:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     22997
           1       0.09      0.28      0.13       386

    accuracy                           0.94     23383
   macro avg       0.54      0.62      0.55     23383
weighted avg       0.97      0.94      0.95     23383

Матрица ошибок на тестовой (out-of-sample) выборке:
[[21855  1142]
 [  277   109]]


2024-12-12 13:38:03,887 - INFO - ROC-AUC на out-of-time выборке: 0.7603



Отчёт по классификации на out-of-time выборке:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     25567
           1       0.02      0.10      0.03       114

    accuracy                           0.97     25681
   macro avg       0.51      0.54      0.51     25681
weighted avg       0.99      0.97      0.98     25681

Матрица ошибок на out-of-time выборке:
[[24989   578]
 [  103    11]]


2024-12-12 13:38:03,909 - INFO - ROC-AUC на out-of-time 2 выборке: 0.7684



Отчёт по классификации на out-of-time 2 выборке:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     16525
           1       0.06      0.27      0.09       177

    accuracy                           0.94     16702
   macro avg       0.52      0.61      0.53     16702
weighted avg       0.98      0.94      0.96     16702

Матрица ошибок на out-of-time 2 выборке:
[[15726   799]
 [  130    47]]


2024-12-12 13:38:03,935 - INFO - ROC-AUC на Development Final выборке: 0.7789



Отчёт по классификации на Development Final выборке:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     76654
           1       0.08      0.27      0.13      1288

    accuracy                           0.94     77942
   macro avg       0.53      0.61      0.55     77942
weighted avg       0.97      0.94      0.95     77942

Матрица ошибок на Development Final выборке:
[[72803  3851]
 [  944   344]]


In [149]:
# Check model performance 
from sklearn.metrics import (accuracy_score, confusion_matrix, roc_auc_score,   
                             precision_score, f1_score, recall_score)  
from scipy.stats import ks_2samp  # Importing ks_2samp  

# Initialize a dictionary to hold all metrics  
metrics = {  
    'Dataset': [],  
    'Accuracy': [],  
    'Confusion Matrix': [],  
    'ROC AUC': [],  
    'Gini': [],  
    'KS Statistic': [],  
    'Precision': [],  
    'Recall': [],  
    'F1 Score': [],  
}  

def calculate_ks(y_true, y_scores):  
    # Calculate the KS statistic using ks_2samp  
    return ks_2samp(y_scores[y_true == 1], y_scores[y_true == 0]).statistic  

def calculate_metrics(model, X, y, dataset_name):  
    # Predictions and probabilities  
    y_pred = model.predict(X)  
    y_pred_proba = model.predict_proba(X)[:, 1]  

    # Calculate metrics  
    accuracy = accuracy_score(y, y_pred)  
    conf_matrix = confusion_matrix(y, y_pred)  
    precision = precision_score(y, y_pred, zero_division=0)  
    recall = recall_score(y, y_pred, zero_division=0)  
    f1 = f1_score(y, y_pred, zero_division=0)  
    roc_auc = roc_auc_score(y, y_pred_proba)  
    gini = 2 * roc_auc - 1  
    ks = calculate_ks(y, y_pred_proba)  

    # Append metrics to the dictionary  
    metrics['Dataset'].append(dataset_name)  
    metrics['Accuracy'].append(accuracy)  
    metrics['Confusion Matrix'].append(conf_matrix)  
    metrics['ROC AUC'].append(roc_auc)  
    metrics['Gini'].append(gini)  
    metrics['KS Statistic'].append(ks)  
    metrics['Precision'].append(precision) 
    metrics['Recall'].append(recall)  
    metrics['F1 Score'].append(f1)  
    
    
def return_conf_matrix(model, X, y, dataset_name):
    y_pred = model.predict(X)  
    y_pred_proba = model.predict_proba(X)[:, 1]  
    
    conf_matrix = confusion_matrix(y, y_pred)  

    return conf_matrix

# Call the function for each dataset  
calculate_metrics(model_selected, X_train_selected_scaled, y_train_selected, 'TRAIN')  
calculate_metrics(model_selected, X_test_selected_scaled, y_test_selected, 'TEST')  
calculate_metrics(model_selected, X_oot_selected_scaled, y_oot_selected, 'OOT')  
calculate_metrics(model_selected, X_oot2_selected_scaled, y_oot2_selected, 'OOT2')  
calculate_metrics(model_selected, X_dev_final_selected_scaled, y_dev_final_selected, 'FULL')  

# Create DataFrame from metrics  
metrics_df = pd.DataFrame(metrics)  

# Display the DataFrame  
metrics_df

,Dataset,Accuracy,Confusion Matrix,ROC AUC,Gini,KS Statistic,Precision,Recall,F1 Score
0,TRAIN,0.938122,"[[50948, 2709], [667, 235]]",0.777454,0.554908,0.422430,0.079823,0.260532,0.122205
1,TEST,0.939315,"[[21855, 1142], [277, 109]]",0.782004,0.564008,0.437296,0.087130,0.282383,0.133170
2,OOT,0.973482,"[[24989, 578], [103, 11]]",0.760254,0.520507,0.412752,0.018676,0.096491,0.031294
3,OOT2,0.944378,"[[15726, 799], [130, 47]]",0.768429,0.536859,0.392893,0.055556,0.265537,0.091887
4,FULL,0.938480,"[[72803, 3851], [944, 344]]",0.778883,0.557766,0.424293,0.082002,0.267081,0.125479


In [150]:
print('TRAIN')
print(return_conf_matrix(model_selected, X_train_selected_scaled, y_train_selected, 'TRAIN'))  

print('TEST')
print(return_conf_matrix(model_selected, X_test_selected_scaled, y_test_selected, 'TEST'))  

print('OOT')
print(return_conf_matrix(model_selected, X_oot_selected_scaled, y_oot_selected, 'OOT'))

print('OOT2')
print(return_conf_matrix(model_selected, X_oot2_selected_scaled, y_oot2_selected, 'OOT2'))

print('FULL')
print(return_conf_matrix(model_selected, X_dev_final_selected_scaled, y_dev_final_selected, 'FULL'))

TRAIN
[[50948  2709]
 [  667   235]]
TEST
[[21855  1142]
 [  277   109]]
OOT
[[24989   578]
 [  103    11]]
OOT2
[[15726   799]
 [  130    47]]
FULL
[[72803  3851]
 [  944   344]]


# Визуализация ROC-кривых

In [151]:
print(sc.perf_eva(y_train_selected, model_selected.predict_log_proba(X_train_selected_scaled)[:, 0], title = "train"))
print(sc.perf_eva(y_test_selected, model_selected.predict_log_proba(X_test_selected_scaled)[:, 0], title = "test"))
print(sc.perf_eva(y_oot_selected, model_selected.predict_log_proba(X_oot_selected_scaled)[:, 0], title = "oot"))
print(sc.perf_eva(y_oot2_selected, model_selected.predict_log_proba(X_oot2_selected_scaled)[:, 0], title = "oot2"))
print(sc.perf_eva(y_dev_final_selected, model_selected.predict_log_proba(X_dev_final_selected_scaled)[:, 0], title = "full"))

{'KS': 0.4225, 'AUC': 0.7775, 'Gini': 0.5549, 'pic': <Figure size 1000x600 with 2 Axes>}
{'KS': 0.4373, 'AUC': 0.782, 'Gini': 0.564, 'pic': <Figure size 1000x600 with 2 Axes>}
{'KS': 0.4128, 'AUC': 0.7603, 'Gini': 0.5205, 'pic': <Figure size 1000x600 with 2 Axes>}
{'KS': 0.3929, 'AUC': 0.7684, 'Gini': 0.5369, 'pic': <Figure size 1000x600 with 2 Axes>}
{'KS': 0.4243, 'AUC': 0.7789, 'Gini': 0.5578, 'pic': <Figure size 1000x600 with 2 Axes>}


In [ ]:
# pip install -r requirements.txt

In [ ]:
pip list
